In [37]:
# Configuración inicial y librerías
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime
from pathlib import Path
from scipy.optimize import minimize
from scipy import stats
import requests
import warnings
warnings.filterwarnings('ignore')

# Configuración
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:.4f}' if pd.notna(x) else '')
np.random.seed(42)

print("✅ Configuración completada")
print(f"📊 Sistema listo para análisis financiero")
print(f"🌎 Librerías para datos macroeconómicos disponibles")

✅ Configuración completada
📊 Sistema listo para análisis financiero
🌎 Librerías para datos macroeconómicos disponibles


## 1. Carga y Preparación de Datos

Carga de datos históricos desde Excel y complemento con APIs financieras para crear dataset completo.

In [38]:
# CARGA DE DATOS COMPLETA Y LIMPIA

print("CARGANDO DATASET COMPLETO...")

# 1. CARGAR ARCHIVO EXCEL Y LIMPIAR DATOS
archivo_base = r"c:\Users\trico\OneDrive\UBA\Management Financiero Bursatil\Cartera final\Datos_historicos_de_la_cartera.xlsx"

# Leer datos principales
df_data_completo = pd.read_excel(archivo_base)

# Limpiar y preparar datos
if 'Fecha' in df_data_completo.columns:
    df_data_completo['Fecha'] = pd.to_datetime(df_data_completo['Fecha'])
    
df_data_completo = df_data_completo.ffill().bfill()

# IDENTIFICAR SOLO ACTIVOS ORIGINALES (no columnas de peso ni variables macro)
# ESTO ARREGLA EL BUG: Solo incluir activos verdaderos, excluir columnas calculadas
activos_originales = ['AAPL', 'SPY', 'EWZ', 'CEPU', 'BHIL', 'METRO', 'IBM']
activos_excel = [col for col in df_data_completo.columns 
                 if col in activos_originales and col != 'Fecha']

print(f"✅ Activos VERDADEROS del Excel: {activos_excel}")

# 2. AGREGAR VARIABLES MACROECONÓMICAS
print("\n💱 AGREGANDO DATOS MACROECONÓMICOS...")

# Limpiar variables existentes
for col in ['Tipo_Cambio_ARSUSD', 'Retorno_FX', 'Tasa_PlazoFijo_Diaria']:
    if col in df_data_completo.columns:
        df_data_completo = df_data_completo.drop(columns=[col])
        print(f"🧹 Eliminada columna existente: {col}")

# Variables macro y retornos
for activo in activos_excel:
    col_retorno = f'Retorno_{activo}'
    if col_retorno not in df_data_completo.columns:
        df_data_completo[col_retorno] = df_data_completo[activo].pct_change()

# Tipo de cambio
if 'Tipo_Cambio_ARSUSD' not in df_data_completo.columns:
    try:
        print("🔄 Obteniendo datos históricos del Dólar CCL (ArgentinaDatos)...")
        from datetime import datetime, timedelta
        import requests
        
        # PEDIR TODOS LOS DATOS SIN FECHAS ESPECÍFICAS (funciona mejor)
        print("   • Consultando API sin restricción de fechas...")
        url = "https://api.argentinadatos.com/v1/cotizaciones/dolares/contadoconliqui"
        response = requests.get(url)
        
        if response.status_code == 200:
            ccl_data = response.json()
            print(f"   • API devolvió {len(ccl_data)} registros históricos")
            
            # Convertir a DataFrame
            ccl_df = pd.DataFrame(ccl_data)
            ccl_df['fecha'] = pd.to_datetime(ccl_df['fecha'])
            ccl_df = ccl_df[['fecha', 'venta']].rename(columns={'fecha': 'Fecha', 'venta': 'Tipo_Cambio_ARSUSD'})
            
            # FILTRAR por fechas que necesitamos
            fecha_min = df_data_completo['Fecha'].min()
            fecha_max = df_data_completo['Fecha'].max()
            ccl_df = ccl_df[(ccl_df['Fecha'] >= fecha_min) & (ccl_df['Fecha'] <= fecha_max)]
            
            print(f"   • Filtrado para período {fecha_min.date()} - {fecha_max.date()}: {len(ccl_df)} registros")
            
            # Merge con datos principales
            df_data_completo = pd.merge(df_data_completo, ccl_df, on='Fecha', how='left')
            df_data_completo['Tipo_Cambio_ARSUSD'] = df_data_completo['Tipo_Cambio_ARSUSD'].ffill().bfill()
            df_data_completo['Retorno_FX'] = df_data_completo['Tipo_Cambio_ARSUSD'].pct_change()
            
            print(f"✅ CCL histórico obtenido y filtrado exitosamente")
            print(f"   Período: {df_data_completo['Fecha'].min().date()} → {df_data_completo['Fecha'].max().date()}")
            print(f"   Rango CCL: ${df_data_completo['Tipo_Cambio_ARSUSD'].min():.1f} - ${df_data_completo['Tipo_Cambio_ARSUSD'].max():.1f}")
            print(f"✅ Tipo_Cambio_ARSUSD y Retorno_FX agregados ({df_data_completo['Tipo_Cambio_ARSUSD'].count()} registros)")
            
        else:
            print(f"❌ CCL NO agregado - Código de respuesta: {response.status_code}")
            print(f"   Respuesta: {response.text[:200]}")
            
    except Exception as e:
        print(f"❌ Error obteniendo CCL: {str(e)}")

# Tasas de plazo fijo (BCRA v3) – FIX
if 'Tasa_PlazoFijo_Diaria' not in df_data_completo.columns:
    try:
        print("🔄 Obteniendo Tasa Plazo Fijo (BCRA v3)…")
        import requests
        import urllib3
        urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

        df_data_completo['Fecha'] = pd.to_datetime(df_data_completo['Fecha']).dt.normalize()
        fecha_min = df_data_completo['Fecha'].min().date().isoformat()
        fecha_max = df_data_completo['Fecha'].max().date().isoformat()

        base = "https://api.bcra.gob.ar/estadisticas/v3.0/Monetarias"
        vr = requests.get(base, timeout=20, verify=False)
        vr.raise_for_status()
        variables = vr.json().get('results', [])

        descs = [v for v in variables if v.get('descripcion')]
        cand = [v for v in descs
                if ('plazo' in v['descripcion'].lower() or 'depósito' in v['descripcion'].lower() or 'dep\u00f3sito' in v['descripcion'].lower())
                and ('30 d' in v['descripcion'].lower() or '30 días' in v['descripcion'].lower() or '30 d\u00edas' in v['descripcion'].lower())
                and '% n.a' in v['descripcion'].lower()]

        if not cand:
            cand = [v for v in descs if 'badlar en pesos de bancos privados' in v['descripcion'].lower()]

        if not cand:
            raise Exception("No se encontró idVariable para Plazo Fijo 30 días ni BADLAR.")

        id_pf = cand[0]['idVariable']

        sr = requests.get(f"{base}/{id_pf}", params={
            'desde': f"{fecha_min}T00:00:00",
            'hasta': f"{fecha_max}T23:59:59",
            'limit': 3000,  # Máximo permitido por BCRA
            'offset': 0
        }, timeout=20, verify=False)
        sr.raise_for_status()
        datos = sr.json().get('results', [])

        tasas_df = pd.DataFrame([
            {'Fecha': pd.to_datetime(it['fecha']).normalize(), 'TNA_PlazoFijo': float(it['valor'])}
            for it in datos if it.get('valor') is not None
        ])

        df_data_completo = df_data_completo.merge(tasas_df, on='Fecha', how='left').sort_values('Fecha')
        df_data_completo['TNA_PlazoFijo'] = df_data_completo['TNA_PlazoFijo'].ffill().bfill()
        df_data_completo['Tasa_PlazoFijo_Diaria'] = df_data_completo['TNA_PlazoFijo'] / 36500.0

        print(f"✅ Tasa PF agregada (id {id_pf}) con {df_data_completo['Tasa_PlazoFijo_Diaria'].notna().sum()} registros")
        print(f"   Rango TNA: {df_data_completo['TNA_PlazoFijo'].min():.2f}% - {df_data_completo['TNA_PlazoFijo'].max():.2f}%")

    except Exception as e:
        print(f"❌ Error obteniendo Tasa PF (v3): {e}")

# 3. CALCULAR PESOS REALES DE LA CARTERA POR DÍA - SOLO ACTIVOS VERDADEROS
# ARREGLO DEL BUG: Solo usar activos originales, no columnas de peso existentes
activos_cartera = [col for col in activos_excel if not col.startswith(('Retorno_', 'Tipo_', 'Tasa_', 'Peso_'))]

if len(activos_cartera) > 0:
    print(f"\n💼 CALCULANDO PESOS REALES DE CARTERA:")
    print(f"   • Activos VERDADEROS en cartera: {activos_cartera}")
    
    # LIMPIAR COLUMNAS DE PESO DUPLICADAS ANTES DE RECREAR
    columnas_peso_existentes = [col for col in df_data_completo.columns if col.startswith('Peso_')]
    if columnas_peso_existentes:
        print(f"   🧹 Eliminando {len(columnas_peso_existentes)} columnas de peso duplicadas...")
        df_data_completo = df_data_completo.drop(columns=columnas_peso_existentes)
    
    # Calcular valor total y pesos
    df_final = df_data_completo.copy()
    df_final['Valor_Total_Cartera'] = df_final[activos_cartera].sum(axis=1)
    
    # Calcular peso de cada activo por día
    for activo in activos_cartera:
        col_peso = f'Peso_{activo}'
        df_final[col_peso] = df_final[activo] / df_final['Valor_Total_Cartera']
    
    # Crear diccionario de cartera real con pesos del último día disponible
    ultimo_dia = df_final.dropna(subset=['Valor_Total_Cartera']).iloc[-1]
    cartera_real_weights = {}
    
    print(f"   • Pesos al {ultimo_dia['Fecha'].strftime('%d/%m/%Y')}:")
    for activo in activos_cartera:
        peso = ultimo_dia[f'Peso_{activo}']
        if not pd.isna(peso):
            cartera_real_weights[activo] = peso
            print(f"     - {activo}: {peso:.4f} ({peso*100:.2f}%)")
    
    # Guardar cartera real como variable global
    cartera_real = {
        'weights': cartera_real_weights,
        'activos': list(cartera_real_weights.keys()),
        'tipo': 'Pesos por Valor Real',
        'fecha_calculo': ultimo_dia['Fecha']
    }
    
    globals()['cartera_real'] = cartera_real
    globals()['cartera_real_weights'] = cartera_real_weights
    print(f"   ✅ Cartera real guardada en variable global")

# 4. Finalizar dataset y LIMPIAR VARIABLES DUPLICADAS
df_data_completo = df_final.sort_values('Fecha').reset_index(drop=True)

# LIMPIAR TODAS LAS COLUMNAS DUPLICADAS DE TASAS
print("🧹 Limpiando columnas duplicadas de tasas...")

# Eliminar columnas con sufijos _x, _y (resultado de merges mal hechos)
cols_to_drop = []
for col in df_data_completo.columns:
    if col.endswith('_x') or col.endswith('_y'):
        cols_to_drop.append(col)
        print(f"   • Eliminando columna duplicada: {col}")

if cols_to_drop:
    df_data_completo = df_data_completo.drop(columns=cols_to_drop)

# Si hay múltiples columnas TNA_PlazoFijo, mantener solo la última
tna_cols = [col for col in df_data_completo.columns if col.startswith('TNA_PlazoFijo') and not col.endswith(('_x', '_y'))]
if len(tna_cols) > 1:
    print(f"   • Múltiples columnas TNA encontradas: {tna_cols}")
    # Mantener solo la última columna TNA_PlazoFijo y eliminar las otras
    for col in tna_cols[:-1]:
        df_data_completo = df_data_completo.drop(columns=[col])
        print(f"   • Eliminando columna TNA duplicada: {col}")

# Eliminar columnas Tasa_PlazoFijo_Diaria duplicadas si existen múltiples
tasa_cols = [col for col in df_data_completo.columns if col.startswith('Tasa_PlazoFijo_Diaria')]
if len(tasa_cols) > 1:
    for col in tasa_cols[:-1]:
        df_data_completo = df_data_completo.drop(columns=[col])
        print(f"   • Eliminando columna Tasa duplicada: {col}")

# SEPARAR ACTIVOS VERDADEROS DE VARIABLES MACRO
activos_verdaderos = [col for col in df_data_completo.columns 
                     if col not in ['Fecha', 'Valor_Total_Cartera']
                     and not col.startswith(('Tipo_', 'Tasa_', 'Peso_', 'TNA_', 'Retorno_FX'))
                     and col in ['AAPL', 'SPY', 'EWZ', 'CEPU', 'BHIL', 'METRO', 'IBM']]  # Solo activos originales

# VARIABLES MACRO REALES (NO incluir retornos de activos individuales)
variables_macro = [col for col in df_data_completo.columns 
                  if col in ['Tipo_Cambio_ARSUSD', 'Retorno_FX', 'TNA_PlazoFijo', 'Tasa_PlazoFijo_Diaria']]

# VARIABLES DE RETORNOS DE ACTIVOS (clasificación separada)
retornos_activos = [col for col in df_data_completo.columns 
                   if col.startswith('Retorno_') and col != 'Retorno_FX']

pesos_cols = [col for col in df_data_completo.columns if col.startswith('Peso_')]

print(f"\n🎯 DATASET FINAL CORREGIDO Y LIMPIO:")
print(f"   • Activos VERDADEROS: {len(activos_verdaderos)} {activos_verdaderos}")
print(f"   • Variables MACRO (reales): {len(variables_macro)} {variables_macro}")
print(f"   • Retornos de activos: {len(retornos_activos)} {retornos_activos}")
print(f"   • Columnas de pesos: {len(pesos_cols)}")
print(f"   • Total columnas: {len(df_data_completo.columns)}")
print(f"   • Registros: {len(df_data_completo)}")
print(f"   • Período: {df_data_completo['Fecha'].min().strftime('%d/%m/%Y')} → {df_data_completo['Fecha'].max().strftime('%d/%m/%Y')}")

print(f"\n📊 VERIFICACIÓN DE DATOS:")
for activo in activos_verdaderos:
    valid_count = df_data_completo[activo].notna().sum()
    print(f"   📈 Activo {activo:20s}: {valid_count:3d} registros válidos")

for macro_var in variables_macro:
    valid_count = df_data_completo[macro_var].notna().sum()
    print(f"   🌍 Macro {macro_var:20s}: {valid_count:3d} registros válidos")

for ret_var in retornos_activos:
    valid_count = df_data_completo[ret_var].notna().sum()
    print(f"   📊 Retorno {ret_var:18s}: {valid_count:3d} registros válidos")

# Guardar variables globales LIMPIAS
globals()['activos_verdaderos'] = activos_verdaderos
globals()['variables_macro'] = variables_macro
globals()['retornos_activos'] = retornos_activos

# ACTUALIZAR ARCHIVO EXCEL CON VARIABLES MACRO Y PESOS DE CARTERA
try:
    variables_nuevas = [col for col in df_data_completo.columns if col.startswith(('Retorno_', 'Tipo_', 'Tasa_', 'Peso_', 'Valor_', 'TNA_'))]
    
    if variables_nuevas:
        archivo_datos = r"c:\Users\trico\OneDrive\UBA\Management Financiero Bursatil\Cartera final\Datos_historicos_de_la_cartera.xlsx"
        
        with pd.ExcelWriter(archivo_datos, engine='openpyxl') as writer:
            df_data_completo.to_excel(writer, sheet_name='Datos', index=False)
        
        print(f"\n💾 ACTUALIZADO: Excel con nuevas variables:")
        print(f"   🌍 Variables MACRO (reales): {len(variables_macro)} columnas")
        print(f"   📊 Retornos de activos: {len(retornos_activos)} columnas")
        print(f"   💼 Variables de cartera: {len(pesos_cols)} columnas")
        print(f"   ✅ Formato de fecha preservado")
        print(f"   💰 Pesos de cartera disponibles para análisis posteriores")

        print(f"\n🔧 Variables globales creadas:")
        print(f"   • activos_verdaderos: {activos_verdaderos}")
        print(f"   • variables_macro (reales): {variables_macro}")
        print(f"   • retornos_activos: {retornos_activos}")
        print(f"   • cartera_real_weights: {len(cartera_real_weights)} activos")

except Exception as e:
    print(f"❌ Error actualizando Excel: {e}")
    
    print(f"\n💾 ACTUALIZADO: Excel con nuevas variables:")
    print(f"   🌍 Variables MACRO (reales): {len(variables_macro)} columnas")
    print(f"   📊 Retornos de activos: {len(retornos_activos)} columnas")
    print(f"   💼 Variables de cartera: {len(pesos_cols)} columnas")
    print(f"   ✅ Formato de fecha preservado")
    print(f"   💰 Pesos de cartera disponibles para análisis posteriores")

    print(f"\n🔧 Variables globales creadas:")
    print(f"   • activos_verdaderos: {activos_verdaderos}")
    print(f"   • variables_macro (reales): {variables_macro}")
    print(f"   • retornos_activos: {retornos_activos}")
    print(f"   • cartera_real_weights: {len(cartera_real_weights)} activos")

# CREAR VARIABLES GLOBALES PARA USO POSTERIOR
activos_verdaderos = activos_cartera.copy()
activos_reales = activos_cartera.copy()
globals()['activos_verdaderos'] = activos_verdaderos
globals()['activos_reales'] = activos_reales

print(f"\n🔧 Variables globales creadas:")
print(f"   • activos_verdaderos: {activos_verdaderos}")
print(f"   • cartera_real_weights: {len(cartera_real_weights)} activos")

CARGANDO DATASET COMPLETO...
✅ Activos VERDADEROS del Excel: ['AAPL', 'SPY', 'EWZ', 'CEPU', 'BHIL', 'METRO', 'IBM']

💱 AGREGANDO DATOS MACROECONÓMICOS...
🧹 Eliminada columna existente: Tipo_Cambio_ARSUSD
🧹 Eliminada columna existente: Retorno_FX
🔄 Obteniendo datos históricos del Dólar CCL (ArgentinaDatos)...
   • Consultando API sin restricción de fechas...
   • API devolvió 4464 registros históricos
   • Filtrado para período 2025-01-03 - 2025-09-02: 243 registros
✅ CCL histórico obtenido y filtrado exitosamente
   Período: 2025-01-03 → 2025-09-02
   Rango CCL: $1137.0 - $1429.3
✅ Tipo_Cambio_ARSUSD y Retorno_FX agregados (172 registros)
🔄 Obteniendo Tasa Plazo Fijo (BCRA v3)…
   • API devolvió 4464 registros históricos
   • Filtrado para período 2025-01-03 - 2025-09-02: 243 registros
✅ CCL histórico obtenido y filtrado exitosamente
   Período: 2025-01-03 → 2025-09-02
   Rango CCL: $1137.0 - $1429.3
✅ Tipo_Cambio_ARSUSD y Retorno_FX agregados (172 registros)
🔄 Obteniendo Tasa Plazo Fi

## 2. Análisis de Riesgo y Performance Avanzado

Cálculo completo de métricas financieras: Sharpe, Sortino, Alpha, Beta, Treynor, Kelly, VaR, CVaR y más.

In [39]:
class RiskCalculator:
    def __init__(self, risk_free_rate=0.02, benchmark='MERVAL'):
        self.rf_rate = risk_free_rate / 252
        self.annual_rf = risk_free_rate
        self.benchmark = benchmark  # AGREGAR ATRIBUTO BENCHMARK
    
    def set_dynamic_risk_free_rate(self, df):
        if 'Tasa_PlazoFijo_Diaria' in df.columns:
            self.dynamic_rf = df['Tasa_PlazoFijo_Diaria'].mean()
            self.annual_rf = self.dynamic_rf * 365
            self.rf_rate = self.dynamic_rf
            print(f"✅ Tasa libre de riesgo: {self.annual_rf:.2%} anual")
    
    def calculate_fx_metrics(self, returns, df):
        fx_metrics = {'fx_correlation': None, 'fx_beta': None, 'crisis_volatility_ratio': None}
        
        if 'Retorno_FX' in df.columns:
            fx_returns = df['Retorno_FX'].dropna()
            common_dates = returns.index.intersection(fx_returns.index)
            if len(common_dates) > 30:
                aligned_returns = returns.loc[common_dates]
                aligned_fx = fx_returns.loc[common_dates]
                
                correlation = np.corrcoef(aligned_returns, aligned_fx)[0, 1]
                fx_metrics['fx_correlation'] = correlation if not np.isnan(correlation) else 0
                
                # Beta FX (sensibilidad al tipo de cambio)
                covariance = np.cov(aligned_returns, aligned_fx)[0, 1]
                fx_variance = np.var(aligned_fx)
                fx_metrics['fx_beta'] = covariance / fx_variance if fx_variance > 0 else 0
                
                # Volatilidad durante crisis cambiarias (retorno FX > 2 std)
                crisis_threshold = aligned_fx.mean() + 2 * aligned_fx.std()
                crisis_periods = aligned_fx > crisis_threshold
                if crisis_periods.any():
                    crisis_vol = aligned_returns[crisis_periods].std() * np.sqrt(252)
                    normal_vol = aligned_returns[~crisis_periods].std() * np.sqrt(252)
                    fx_metrics['crisis_volatility_ratio'] = crisis_vol / normal_vol if normal_vol > 0 else 1
                else:
                    fx_metrics['crisis_volatility_ratio'] = 1
            else:
                fx_metrics = {'fx_correlation': 0, 'fx_beta': 0, 'crisis_volatility_ratio': 1}
        else:
            fx_metrics = {'fx_correlation': None, 'fx_beta': None, 'crisis_volatility_ratio': None}
        
        return fx_metrics
    
    def get_benchmark_returns(self, df, asset_dates=None):
        """Obtiene retornos del benchmark"""
        if self.benchmark not in df.columns:
            # Crear benchmark sintético si no existe - USAR SOLO ACTIVOS VERDADEROS
            available_assets = [col for col in df.columns 
                              if not col.startswith(('Peso_', 'Retorno_', 'Tipo_', 'Tasa_', 'Fecha', 'Valor_Total'))]
            if len(available_assets) >= 3:
                market_prices = df[available_assets[:3]].mean(axis=1, skipna=True)
                market_returns = market_prices.pct_change().dropna()
                return market_returns.values
        else:
            benchmark_prices = df[self.benchmark].dropna()
            benchmark_returns = benchmark_prices.pct_change().dropna()
            return benchmark_returns.values
        return None
    
    def calculate_alpha_beta(self, asset_returns, benchmark_returns):
        """Calcula Alpha y Beta usando numpy (evitamos sklearn)"""
        if len(asset_returns) != len(benchmark_returns):
            min_len = min(len(asset_returns), len(benchmark_returns))
            asset_returns = asset_returns[:min_len]
            benchmark_returns = benchmark_returns[:min_len]
        
        # Filtrar NaN
        mask = ~(np.isnan(asset_returns) | np.isnan(benchmark_returns))
        asset_returns = asset_returns[mask]
        benchmark_returns = benchmark_returns[mask]
        
        if len(asset_returns) < 30:
            return None, None, None
        
        try:
            # Regresión lineal simple usando numpy
            # Beta = Cov(R_asset, R_market) / Var(R_market)
            covariance = np.cov(asset_returns, benchmark_returns)[0, 1]
            market_variance = np.var(benchmark_returns)
            
            if market_variance == 0:
                return None, None, None
            
            beta = covariance / market_variance
            
            # Alpha = R_asset_mean - Beta * R_market_mean
            alpha_daily = np.mean(asset_returns) - beta * np.mean(benchmark_returns)
            alpha_annual = alpha_daily * 252
            
            # R-squared
            correlation = np.corrcoef(asset_returns, benchmark_returns)[0, 1]
            r_squared = correlation ** 2 if not np.isnan(correlation) else 0
            
            return alpha_annual, beta, r_squared
            
        except Exception:
            return None, None, None
    
    def calculate_advanced_metrics(self, returns, df=None, asset_name=None):
        """Calcula métricas avanzadas adicionales"""
        metrics = {}
        
        # Inicializar con None para asegurar consistencia
        metrics['alpha'] = None
        metrics['beta'] = None
        metrics['r_squared'] = None
        metrics['treynor_ratio'] = None
        metrics['information_ratio'] = None
        metrics['tracking_error'] = None
        
        # Treynor Ratio y métricas vs benchmark
        if df is not None:
            benchmark_returns = self.get_benchmark_returns(df)
            if benchmark_returns is not None and len(benchmark_returns) > 30:
                try:
                    alpha, beta, r_squared = self.calculate_alpha_beta(returns.values, benchmark_returns)
                    
                    if alpha is not None and beta is not None and r_squared is not None:
                        metrics['alpha'] = alpha
                        metrics['beta'] = beta
                        metrics['r_squared'] = r_squared
                        
                        # Treynor Ratio
                        if beta is not None and abs(beta) > 0.001:  # Evitar división por cero
                            excess_return = returns.mean() * 252 - self.annual_rf
                            metrics['treynor_ratio'] = excess_return / beta
                        
                        # Information Ratio y Tracking Error
                        min_len = min(len(returns), len(benchmark_returns))
                        if min_len > 30:
                            asset_trim = returns.values[:min_len]
                            bench_trim = benchmark_returns[:min_len]
                            
                            # Filtrar NaNs
                            mask = ~(np.isnan(asset_trim) | np.isnan(bench_trim))
                            if mask.sum() > 30:
                                excess_returns = asset_trim[mask] - bench_trim[mask]
                                tracking_error = np.std(excess_returns) * np.sqrt(252)
                                metrics['tracking_error'] = tracking_error
                                
                                if tracking_error > 0:
                                    metrics['information_ratio'] = (np.mean(excess_returns) * 252) / tracking_error
                                    
                except Exception as e:
                    print(f"⚠️  Error calculando métricas benchmark para {asset_name}: {str(e)[:50]}")
        
        # Kelly Criterion
        positive_returns = returns[returns > 0]
        negative_returns = returns[returns < 0]
        
        if len(positive_returns) > 0 and len(negative_returns) > 0:
            avg_win = positive_returns.mean()
            avg_loss = abs(negative_returns.mean())
            win_rate = len(positive_returns) / len(returns)
            
            if avg_loss > 0:
                b = avg_win / avg_loss
                kelly_f = (b * win_rate - (1 - win_rate)) / b
                metrics['kelly_criterion'] = max(0, min(kelly_f, 1))
            else:
                metrics['kelly_criterion'] = None
        else:
            metrics['kelly_criterion'] = None
        
        # Calmar Ratio
        cumulative = (1 + returns).cumprod()
        running_max = cumulative.cummax()
        drawdown = (cumulative - running_max) / running_max
        max_dd = drawdown.min()
        
        if max_dd < 0:
            annual_return = returns.mean() * 252
            metrics['calmar_ratio'] = annual_return / abs(max_dd)
        else:
            metrics['calmar_ratio'] = None
        
        # Ulcer Index
        drawdown_squared = (drawdown * 100) ** 2
        metrics['ulcer_index'] = np.sqrt(drawdown_squared.mean())
        
        return metrics
    
    def calculate_metrics(self, returns, df=None, asset_name=None):
        """Calcula métricas completas de riesgo y rendimiento"""
        if len(returns) < 30:
            return None
        
        # Limpiar datos
        returns = returns.dropna()
        returns = returns.replace([np.inf, -np.inf], np.nan).dropna()
        
        if len(returns) < 30:
            return None
        
        # Métricas básicas
        annual_return = returns.mean() * 252
        volatility = returns.std() * np.sqrt(252)
        
        # Sharpe Ratio
        excess_return = annual_return - self.annual_rf
        sharpe = excess_return / volatility if volatility > 0 else 0
        
        # Sortino Ratio
        negative_returns = returns[returns < 0]
        downside_deviation = negative_returns.std() * np.sqrt(252) if len(negative_returns) > 0 else volatility
        sortino = excess_return / downside_deviation if downside_deviation > 0 else 0
        
        # VaR y CVaR
        var_95 = np.percentile(returns, 5)
        cvar_95 = returns[returns <= var_95].mean()
        
        # Maximum Drawdown
        cumulative = (1 + returns).cumprod()
        running_max = cumulative.cummax()
        drawdown = (cumulative - running_max) / running_max
        max_dd = drawdown.min()
        
        # Métricas básicas
        basic_metrics = {
            'rendimiento_anual': annual_return,
            'volatilidad': volatility,
            'sharpe_ratio': sharpe,
            'sortino_ratio': sortino,
            'var_95': var_95,
            'cvar_95': cvar_95,
            'max_drawdown': max_dd,
            'skewness': returns.skew(),
            'kurtosis': returns.kurtosis()
        }
        
        # Métricas avanzadas
        advanced_metrics = self.calculate_advanced_metrics(returns, df, asset_name)
        
        # Combinar ambos diccionarios
        basic_metrics.update(advanced_metrics)
        
        # AGREGAR MÉTRICAS FX si hay datos macro disponibles
        if df is not None:
            fx_metrics = self.calculate_fx_metrics(returns, df)
            basic_metrics.update(fx_metrics)
        
        return basic_metrics

def analizar_activos(df, activos):
    """Analiza todos los activos y retorna métricas completas"""
    calculator = RiskCalculator()
    
    # CONFIGURAR TASA LIBRE DE RIESGO DINÁMICA
    calculator.set_dynamic_risk_free_rate(df)
    
    resultados = []
    
    for i, activo in enumerate(activos):
        precios = df[activo].dropna()
        if len(precios) > 30:
            returns = precios.pct_change().dropna()
            metricas = calculator.calculate_metrics(returns, df, activo)
            if metricas:
                metricas['activo'] = activo
                metricas['observaciones'] = len(returns)
                resultados.append(metricas)
                
                # Mostrar progreso incluyendo métricas FX
                if i < 5:
                    alpha_str = f"α={metricas['alpha']:.4f}" if metricas.get('alpha') else "α=N/A"
                    beta_str = f"β={metricas['beta']:.3f}" if metricas.get('beta') else "β=N/A"
                    fx_corr_str = f"FX_ρ={metricas.get('fx_correlation', 0):.3f}" if metricas.get('fx_correlation') is not None else "FX_ρ=N/A"
                    print(f"   ✅ {activo}: Sharpe={metricas['sharpe_ratio']:.3f}, {alpha_str}, {beta_str}, {fx_corr_str}")
    
    if not resultados:
        return pd.DataFrame()
    
    df_metricas = pd.DataFrame(resultados)
    
    # Reordenar columnas por importancia
    basic_cols = ['activo', 'observaciones', 'rendimiento_anual', 'volatilidad', 
                 'sharpe_ratio', 'sortino_ratio', 'max_drawdown']
    advanced_cols = ['alpha', 'beta', 'r_squared', 'treynor_ratio', 'information_ratio', 
                    'tracking_error', 'kelly_criterion', 'calmar_ratio', 'ulcer_index']
    risk_cols = ['var_95', 'cvar_95', 'skewness', 'kurtosis']
    
    # Verificar qué columnas existen
    available_cols = []
    for col_group in [basic_cols, advanced_cols, risk_cols]:
        available_cols.extend([col for col in col_group if col in df_metricas.columns])
    
    # Agregar columnas restantes
    remaining_cols = [col for col in df_metricas.columns if col not in available_cols]
    all_cols = available_cols + remaining_cols
    
    df_metricas = df_metricas[all_cols]
    
    return df_metricas

# Calcular métricas usando el dataset completo con variables macro
# USAR LA VARIABLE GLOBAL activos_verdaderos YA DEFINIDA CORRECTAMENTE
# NO redefinir aquí para evitar incluir TNA_PlazoFijo

print(f"🔬 Iniciando análisis de {len(activos_verdaderos)} ACTIVOS VERDADEROS:")
for i, activo in enumerate(activos_verdaderos):
    print(f"   {i+1}. {activo}")

print(f"\n🔬 Iniciando análisis completo de métricas de riesgo...")
df_metricas = analizar_activos(df_data_completo, activos_verdaderos)

# Mostrar resultados organizados
if not df_metricas.empty:
    df_display = df_metricas.copy()
    
    # Formatear porcentajes
    percentage_cols = ['rendimiento_anual', 'volatilidad', 'max_drawdown', 'var_95', 'cvar_95', 'alpha', 'tracking_error']
    for col in percentage_cols:
        if col in df_display.columns:
            df_display[col] = (df_display[col] * 100).round(2)
    
    # Formatear ratios
    ratio_cols = ['sharpe_ratio', 'sortino_ratio', 'treynor_ratio', 'information_ratio', 
                  'calmar_ratio', 'kelly_criterion', 'beta', 'r_squared', 'ulcer_index']
    for col in ratio_cols:
        if col in df_display.columns:
            df_display[col] = df_display[col].round(3)
    
    print(f"\n📋 REPORTE COMPLETO DE MÉTRICAS (n={len(df_metricas)} activos)")
    print("="*90)
    
    # Tabla 1: Métricas principales
    main_cols = ['activo', 'rendimiento_anual', 'volatilidad', 'sharpe_ratio', 'max_drawdown']
    available_main = [col for col in main_cols if col in df_display.columns]
    if available_main:
        print("\n🔸 PERFORMANCE Y RIESGO BÁSICO:")
        display(df_display[available_main])
    
    # Tabla 2: Métricas vs Benchmark (Alpha, Beta, etc.)
    benchmark_cols = ['activo', 'alpha', 'beta', 'r_squared', 'treynor_ratio', 'information_ratio', 'tracking_error']
    available_benchmark = [col for col in benchmark_cols if col in df_display.columns]
    
    # Solo mostrar si hay al menos algunos valores no nulos
    has_benchmark_data = any(df_display[col].notna().any() for col in available_benchmark[1:] if col in df_display.columns)
    if available_benchmark and has_benchmark_data:
        print("\n🔸 MÉTRICAS vs BENCHMARK:")
        display(df_display[available_benchmark])
    
    # Tabla 3: Métricas FX (nuevas)
    fx_cols = ['activo', 'fx_correlation', 'fx_beta', 'crisis_volatility_ratio']
    available_fx = [col for col in fx_cols if col in df_display.columns]
    has_fx_data = any(df_display[col].notna().any() for col in available_fx[1:] if col in df_display.columns)
    if available_fx and has_fx_data:
        print("\n💱 MÉTRICAS DE RIESGO CAMBIARIO (CCL):")
        # Formatear las nuevas métricas FX
        fx_display = df_display[available_fx].copy()
        for col in ['fx_correlation', 'fx_beta']:
            if col in fx_display.columns:
                fx_display[col] = fx_display[col].round(3)
        if 'crisis_volatility_ratio' in fx_display.columns:
            fx_display['crisis_volatility_ratio'] = fx_display['crisis_volatility_ratio'].round(2)
        display(fx_display)
    
    # Tabla 4: Métricas avanzadas y especializadas
    advanced_cols = ['activo', 'sortino_ratio', 'kelly_criterion', 'calmar_ratio', 'ulcer_index']
    available_advanced = [col for col in advanced_cols if col in df_display.columns]
    if available_advanced:
        print("\n🔸 MÉTRICAS AVANZADAS:")
        display(df_display[available_advanced])
    
    # Análisis de Top Performers
    print("\n🏆 TOP PERFORMERS:")
    print("─"*60)
    
    # Mejor Sharpe Ratio
    if 'sharpe_ratio' in df_metricas.columns:
        mejor_sharpe = df_metricas.loc[df_metricas['sharpe_ratio'].idxmax()]
        print(f"📈 Mejor Sharpe Ratio: {mejor_sharpe['activo']} ({mejor_sharpe['sharpe_ratio']:.3f})")
    
    # Mejor Alpha (si disponible)
    alpha_data = df_metricas.dropna(subset=['alpha']) if 'alpha' in df_metricas.columns else pd.DataFrame()
    if not alpha_data.empty:
        mejor_alpha = alpha_data.loc[alpha_data['alpha'].idxmax()]
        print(f"🎯 Mejor Alpha: {mejor_alpha['activo']} ({mejor_alpha['alpha']*100:.2f}%)")
    
    # Mejor Kelly Criterion
    kelly_data = df_metricas.dropna(subset=['kelly_criterion']) if 'kelly_criterion' in df_metricas.columns else pd.DataFrame()
    if not kelly_data.empty:
        mejor_kelly = kelly_data.loc[kelly_data['kelly_criterion'].idxmax()]
        print(f"🎲 Mejor Kelly: {mejor_kelly['activo']} ({mejor_kelly['kelly_criterion']:.3f})")
    
    # Menor riesgo (máximo drawdown más bajo)
    menor_riesgo = df_metricas.loc[df_metricas['max_drawdown'].idxmax()]  # idxmax porque son valores negativos
    print(f"🛡️  Menor Riesgo: {menor_riesgo['activo']} (DD: {menor_riesgo['max_drawdown']*100:.2f}%)")
    
    # CALCULAR RATIOS GLOBALES PONDERADOS PARA LA CARTERA REAL (SI EXISTE)
    try:
        if 'cartera_real' in globals() and cartera_real and 'weights' in cartera_real:
            print("\n" + "="*70)
            print("🏛️  RATIOS GLOBALES DE LA CARTERA (PONDERADOS POR PARTICIPACIÓN)")
            print("="*70)
            
            # Obtener pesos de la cartera real
            weights = cartera_real['weights']
            
            # Filtrar métricas solo para activos en la cartera con peso > 0.1%
            activos_cartera = [activo for activo, peso in weights.items() if peso > 0.001]
            df_metricas_cartera = df_metricas[df_metricas['activo'].isin(activos_cartera)].copy()
            
            if not df_metricas_cartera.empty:
                # Mapear pesos
                peso_map = {activo: peso for activo, peso in weights.items() if peso > 0.001}
                df_metricas_cartera['peso_cartera'] = df_metricas_cartera['activo'].map(peso_map)
                
                # Calcular ratios ponderados
                sharpe_global = (df_metricas_cartera['sharpe_ratio'] * df_metricas_cartera['peso_cartera']).sum()
                sortino_global = (df_metricas_cartera['sortino_ratio'].fillna(0) * df_metricas_cartera['peso_cartera']).sum()
                var_global = (df_metricas_cartera['var_95'].fillna(0) * df_metricas_cartera['peso_cartera']).sum()
                cvar_global = (df_metricas_cartera['cvar_95'].fillna(0) * df_metricas_cartera['peso_cartera']).sum()
                dd_global = (df_metricas_cartera['max_drawdown'].fillna(0) * df_metricas_cartera['peso_cartera']).sum()
                kelly_global = (df_metricas_cartera['kelly_criterion'].fillna(0) * df_metricas_cartera['peso_cartera']).sum()
                alpha_global = (df_metricas_cartera['alpha'].fillna(0) * df_metricas_cartera['peso_cartera']).sum()
                beta_global = (df_metricas_cartera['beta'].fillna(1) * df_metricas_cartera['peso_cartera']).sum()
                
                print(f"📊 Composición: {len(activos_cartera)} activos con participación")
                print(f"🎯 Sharpe Ratio Global:      {sharpe_global:>8.4f}")
                print(f"📈 Sortino Ratio Global:     {sortino_global:>8.4f}")
                print(f"⚠️  VaR 95% Global:           {var_global*100:>7.2f}%")
                print(f"🔻 CVaR 95% Global:          {cvar_global*100:>7.2f}%")
                print(f"📉 Max Drawdown Global:      {dd_global*100:>7.2f}%")
                print(f"🎲 Kelly Criterion Global:   {kelly_global:>8.4f}")
                print(f"⚡ Alpha Global:             {alpha_global:>8.4f}")
                print(f"📊 Beta Global:              {beta_global:>8.4f}")
                
                print(f"\n💡 INTERPRETACIÓN GLOBAL:")
                if sharpe_global > 1.0:
                    print("   • Excelente relación riesgo-rendimiento global")
                elif sharpe_global > 0.5:
                    print("   • Buena relación riesgo-rendimiento global")
                else:
                    print("   • Relación riesgo-rendimiento global aceptable")
                    
                print("="*70)
    except Exception as e:
        print(f"⚠️  No se pudieron calcular ratios globales: {str(e)}")
    
    # ACTUALIZAR EXCEL EXISTENTE CON DATOS CALCULADOS
    try:
        archivo_ratios = r"c:\Users\trico\OneDrive\UBA\Management Financiero Bursatil\Cartera final\Ratios Managment financiero cartera.xlsx"
        
        # Actualizar archivo con datos calculados
        with pd.ExcelWriter(archivo_ratios, engine='openpyxl') as writer:
            # ACTUALIZAR hojas específicas con datos calculados
            # Drawdown
            drawdown_data = df_metricas[['activo', 'max_drawdown', 'ulcer_index', 'calmar_ratio']].copy()
            drawdown_data['max_drawdown'] = drawdown_data['max_drawdown'] * 100  # Convertir a porcentaje
            drawdown_data.columns = ['Activo', 'Max_Drawdown_%', 'Ulcer_Index', 'Calmar_Ratio']
            drawdown_data.to_excel(writer, sheet_name='Drawdown', index=False)
            
            # VaR
            var_data = df_metricas[['activo', 'var_95', 'cvar_95']].copy()
            var_data['var_95'] = var_data['var_95'] * 100  
            var_data['cvar_95'] = var_data['cvar_95'] * 100
            var_data.columns = ['Activo', 'VaR_95%', 'CVaR_95%']
            var_data.to_excel(writer, sheet_name='VaR', index=False)
            
            # VaR cond - Usar CVaR
            cvar_data = df_metricas[['activo', 'cvar_95']].copy()
            cvar_data['cvar_95'] = cvar_data['cvar_95'] * 100
            cvar_data.columns = ['Activo', 'CVaR_95%']
            cvar_data.to_excel(writer, sheet_name='VaR cond', index=False)
            
            # Kelly criterion
            kelly_data_sheet = df_metricas[['activo', 'kelly_criterion']].copy()
            kelly_data_sheet.columns = ['Activo', 'Kelly_Criterion']
            kelly_data_sheet.to_excel(writer, sheet_name='Kelly criterion', index=False)
            
            # Sharpe
            sharpe_data = df_metricas[['activo', 'sharpe_ratio', 'rendimiento_anual', 'volatilidad']].copy()
            sharpe_data['rendimiento_anual'] = sharpe_data['rendimiento_anual'] * 100
            sharpe_data['volatilidad'] = sharpe_data['volatilidad'] * 100
            sharpe_data.columns = ['Activo', 'Sharpe_Ratio', 'Rendimiento_Anual_%', 'Volatilidad_%']
            sharpe_data.to_excel(writer, sheet_name='Sharpe', index=False)
            
            # Sortino
            sortino_data = df_metricas[['activo', 'sortino_ratio', 'rendimiento_anual', 'max_drawdown']].copy()
            sortino_data['rendimiento_anual'] = sortino_data['rendimiento_anual'] * 100
            sortino_data['max_drawdown'] = sortino_data['max_drawdown'] * 100
            sortino_data.columns = ['Activo', 'Sortino_Ratio', 'Rendimiento_Anual_%', 'Max_Drawdown_%']
            sortino_data.to_excel(writer, sheet_name='Sortino', index=False)
            
            # Alpha beta
            alpha_beta_data = df_metricas[['activo', 'alpha', 'beta', 'r_squared', 'treynor_ratio']].copy()
            alpha_beta_data['alpha'] = alpha_beta_data['alpha'] * 100
            alpha_beta_data.columns = ['Activo', 'Alpha_%', 'Beta', 'R_Squared', 'Treynor_Ratio']
            alpha_beta_data.to_excel(writer, sheet_name='Alpha beta', index=False)
        
        print(f"\n💾 ACTUALIZADO: '{archivo_ratios}' - TODAS LAS HOJAS ACTUALIZADAS con datos calculados")
        
    except Exception as e:
        print(f"❌ Error actualizando archivo de ratios: {str(e)}")
    
else:
    print("❌ No se pudieron calcular métricas")

🔬 Iniciando análisis de 7 ACTIVOS VERDADEROS:
   1. AAPL
   2. SPY
   3. EWZ
   4. CEPU
   5. BHIL
   6. METRO
   7. IBM

🔬 Iniciando análisis completo de métricas de riesgo...
✅ Tasa libre de riesgo: 32.85% anual
   ✅ AAPL: Sharpe=-0.328, α=-0.2783, β=1.248, FX_ρ=0.215
   ✅ SPY: Sharpe=0.156, α=-0.0100, β=0.983, FX_ρ=0.180
   ✅ EWZ: Sharpe=1.089, α=0.3011, β=0.870, FX_ρ=0.170
   ✅ CEPU: Sharpe=-1.251, α=-0.5592, β=0.651, FX_ρ=0.055
   ✅ BHIL: Sharpe=-2.052, α=-1.0681, β=0.554, FX_ρ=-0.040

📋 REPORTE COMPLETO DE MÉTRICAS (n=7 activos)

🔸 PERFORMANCE Y RIESGO BÁSICO:
   ✅ AAPL: Sharpe=-0.328, α=-0.2783, β=1.248, FX_ρ=0.215
   ✅ SPY: Sharpe=0.156, α=-0.0100, β=0.983, FX_ρ=0.180
   ✅ EWZ: Sharpe=1.089, α=0.3011, β=0.870, FX_ρ=0.170
   ✅ CEPU: Sharpe=-1.251, α=-0.5592, β=0.651, FX_ρ=0.055
   ✅ BHIL: Sharpe=-2.052, α=-1.0681, β=0.554, FX_ρ=-0.040

📋 REPORTE COMPLETO DE MÉTRICAS (n=7 activos)

🔸 PERFORMANCE Y RIESGO BÁSICO:


,activo,rendimiento_anual,volatilidad,sharpe_ratio,max_drawdown
0,AAPL,20.3900,37.9700,-0.3280,-27.6900
1,SPY,37.0200,26.8200,0.1560,-22.0400
2,EWZ,63.7300,28.3700,1.0890,-17.8900
3,CEPU,-30.7400,50.8300,-1.2510,-35.4200
4,BHIL,-85.3900,57.6300,-2.0520,-51.4200
5,METRO,-66.8600,71.6500,-1.3920,-50.6800
6,IBM,19.7500,31.3200,-0.4180,-19.8200



🔸 MÉTRICAS vs BENCHMARK:


,activo,alpha,beta,r_squared,treynor_ratio,information_ratio,tracking_error
0,AAPL,-27.8300,1.2480,0.7410,-0.1000,-0.9010,20.2800
1,SPY,-1.0000,0.9830,0.9230,0.0420,-0.2200,7.4400
2,EWZ,30.1100,0.8700,0.6450,0.3550,1.4550,17.2300
3,CEPU,-55.9200,0.6510,0.1130,-0.9760,-1.4270,48.6300
4,BHIL,-106.8100,0.5540,0.0630,-2.1340,-2.1820,56.8500
5,METRO,-91.9800,0.6500,0.0560,-1.5350,-1.5070,70.0100
6,IBM,6.4700,0.3430,0.0830,-0.3810,-0.5470,34.5600



💱 MÉTRICAS DE RIESGO CAMBIARIO (CCL):


,activo,fx_correlation,fx_beta,crisis_volatility_ratio
0,AAPL,0.2150,0.2190,NaN
1,SPY,0.1800,0.1290,NaN
2,EWZ,0.1700,0.1290,NaN
3,CEPU,0.0550,0.0740,NaN
4,BHIL,-0.0400,-0.0620,NaN
5,METRO,-0.0930,-0.1780,NaN
6,IBM,-0.0550,-0.0460,NaN



🔸 MÉTRICAS AVANZADAS:


,activo,sortino_ratio,kelly_criterion,calmar_ratio,ulcer_index
0,AAPL,-0.4340,0.0000,0.7360,13.2270
1,SPY,0.1930,0.0390,1.6800,5.7540
2,EWZ,1.6340,0.0650,3.5620,5.0460
3,CEPU,-2.2410,0.0000,-0.8680,22.3880
4,BHIL,-3.5370,0.0000,-1.6610,32.0660
5,METRO,-2.6410,0.0000,-1.3190,28.6790
6,IBM,-0.5920,0.0190,0.9960,8.2200



🏆 TOP PERFORMERS:
────────────────────────────────────────────────────────────
📈 Mejor Sharpe Ratio: EWZ (1.089)
🎯 Mejor Alpha: EWZ (30.11%)
🎲 Mejor Kelly: EWZ (0.065)
🛡️  Menor Riesgo: EWZ (DD: -17.89%)

🏛️  RATIOS GLOBALES DE LA CARTERA (PONDERADOS POR PARTICIPACIÓN)
📊 Composición: 7 activos con participación
🎯 Sharpe Ratio Global:        0.2264
📈 Sortino Ratio Global:       0.3117
⚠️  VaR 95% Global:             -2.71%
🔻 CVaR 95% Global:            -4.45%
📉 Max Drawdown Global:       -22.96%
🎲 Kelly Criterion Global:     0.0364
⚡ Alpha Global:              -0.0155
📊 Beta Global:                0.9911

💡 INTERPRETACIÓN GLOBAL:
   • Relación riesgo-rendimiento global aceptable

💾 ACTUALIZADO: 'c:\Users\trico\OneDrive\UBA\Management Financiero Bursatil\Cartera final\Ratios Managment financiero cartera.xlsx' - TODAS LAS HOJAS ACTUALIZADAS con datos calculados


## 3. Visualizaciones y Análisis de Correlación

Gráficos profesionales para análisis de performance y correlaciones.

In [40]:
def crear_visualizaciones(df, activos):
    """Crear visualizaciones profesionales"""
    print("🔍 Iniciando análisis visual...")
    
    # Verificar que hay datos suficientes
    if df.empty or len(activos) == 0:
        print("❌ No hay datos para visualizar")
        return pd.DataFrame()
    
    # Filtrar solo activos válidos que existen en df
    activos_validos = [activo for activo in activos if activo in df.columns]
    print(f"📊 Analizando {len(activos_validos)} activos: {activos_validos}")
    
    # Preparar matriz de retornos
    print("📈 Calculando retornos...")
    returns_data = []
    
    for activo in activos_validos:
        prices = df[activo].dropna()
        if len(prices) > 30:  # Mínimo 30 observaciones
            returns = prices.pct_change().dropna()
            returns_data.append(returns)
        else:
            print(f"⚠️  {activo}: Insuficientes datos ({len(prices)} obs)")
    
    if not returns_data:
        print("❌ No hay suficientes datos de retornos")
        return pd.DataFrame()
    
    # Crear DataFrame de retornos con fechas alineadas
    returns_matrix = pd.concat(returns_data, axis=1)
    returns_matrix.columns = [activo for activo in activos_validos if activo in df.columns and len(df[activo].dropna()) > 30]
    
    print(f"✅ Matriz de retornos: {returns_matrix.shape[0]} días, {returns_matrix.shape[1]} activos")
    
    # 1. GRÁFICO DE PERFORMANCE NORMALIZADA
    try:
        print("📊 Creando gráfico de performance...")
        
        fig_perf = go.Figure()
        
        for activo in activos_validos:
            if activo in df.columns:
                # Calcular retornos y performance para cada activo por separado
                prices = df[activo].dropna()
                if len(prices) > 30:
                    returns_activo = prices.pct_change().dropna()
                    cumulative = (1 + returns_activo).cumprod() * 100
                    
                    fig_perf.add_trace(go.Scatter(
                        x=cumulative.index,
                        y=cumulative.values,
                        mode='lines',
                        name=activo,
                        connectgaps=False,  # No conectar gaps en los datos
                        hovertemplate=f'%{{x}}<br>{activo}: %{{y:.2f}}<extra></extra>'
                    ))
        
        fig_perf.update_layout(
            title='Performance Acumulada Normalizada (Base 100)',
            xaxis_title='Fecha',
            yaxis_title='Valor Base 100',
            template='plotly_white',
            height=500,
            hovermode='x unified'
        )
        
        # Mostrar y guardar
        fig_perf.show()
        fig_perf.write_image("performance_normalizada.png", width=1000, height=500, scale=2)
        print("✅ Performance guardada: performance_normalizada.png")
        
    except Exception as e:
        print(f"❌ Error en gráfico de performance: {str(e)}")
    
    # 2. MATRIZ DE CORRELACIÓN
    try:
        print("📊 Creando matriz de correlación...")
        
        # Calcular correlaciones con datos alineados
        corr_matrix = returns_matrix.corr()
        
        fig_corr = go.Figure(go.Heatmap(
            z=corr_matrix.values,
            x=corr_matrix.columns,
            y=corr_matrix.index,
            colorscale='RdBu', zmid=0,
            text=corr_matrix.round(2).values,
            texttemplate='%{text}',
            hovertemplate='%{y} vs %{x}: %{z:.3f}<extra></extra>'
        ))
        
        fig_corr.update_layout(
            title='Matriz de Correlación de Retornos',
            template='plotly_white', height=400
        )
        # Mostrar y guardar como imagen estática
        fig_corr.show()
        fig_corr.write_image("matriz_correlacion.png", width=800, height=400, scale=2)
        
        # Estadísticas de correlación
        corr_mean = corr_matrix.mean().mean()
        print(f"📊 Correlación promedio: {corr_mean:.3f}")
        
        # RATIOS GLOBALES PARA ANÁLISIS DE CORRELACIÓN
        cartera_para_correlacion = cartera_real if 'cartera_real' in globals() and cartera_real else None
        
        if cartera_para_correlacion and 'df_metricas' in globals() and not df_metricas.empty:
            print(f"\n🏛️  RATIOS GLOBALES - ANÁLISIS DE CORRELACIÓN:")
            print("-" * 50)
            
            weights = cartera_para_correlacion['weights']
            activos_cartera = [activo for activo, peso in weights.items() if peso > 0.001]
            df_metricas_cartera = df_metricas[df_metricas['activo'].isin(activos_cartera)].copy()
            
            if not df_metricas_cartera.empty:
                peso_map = {activo: peso for activo, peso in weights.items() if peso > 0.001}
                df_metricas_cartera['peso_cartera'] = df_metricas_cartera['activo'].map(peso_map)
                
                # Correlación FX ponderada (si disponible)
                if 'fx_correlation' in df_metricas_cartera.columns:
                    fx_corr_global = (df_metricas_cartera['fx_correlation'].fillna(0) * df_metricas_cartera['peso_cartera']).sum()
                    print(f"💱 Correlación FX Global: {fx_corr_global:>8.3f}")
                
                # Beta ponderado
                beta_global = (df_metricas_cartera['beta'].fillna(1) * df_metricas_cartera['peso_cartera']).sum()
                print(f"📊 Beta Global:          {beta_global:>8.3f}")
                
                # Interpretación
                if abs(fx_corr_global) > 0.3:
                    print("   ⚠️ Alta exposición al riesgo cambiario")
                elif abs(fx_corr_global) < 0.1:
                    print("   ✅ Baja exposición al riesgo cambiario")
                else:
                    print("   📊 Exposición moderada al riesgo cambiario")
        
        return returns_matrix
    
    except Exception as e:
        print(f"❌ Error en matriz de correlación: {str(e)}")
        return returns_matrix if 'returns_matrix' in locals() else pd.DataFrame()

# Crear visualizaciones - CORREGIR VARIABLES
print("📊 Generando visualizaciones...")
returns_data = crear_visualizaciones(df_data_completo, activos_verdaderos)

📊 Generando visualizaciones...
🔍 Iniciando análisis visual...
📊 Analizando 7 activos: ['AAPL', 'SPY', 'EWZ', 'CEPU', 'BHIL', 'METRO', 'IBM']
📈 Calculando retornos...
✅ Matriz de retornos: 171 días, 7 activos
📊 Creando gráfico de performance...


✅ Performance guardada: performance_normalizada.png
📊 Creando matriz de correlación...


📊 Correlación promedio: 0.467

🏛️  RATIOS GLOBALES - ANÁLISIS DE CORRELACIÓN:
--------------------------------------------------
💱 Correlación FX Global:    0.176
📊 Beta Global:             0.991
   📊 Exposición moderada al riesgo cambiario


## 4. Optimización de Cartera

Cálculo de carteras óptimas usando teoría moderna de portafolios.

In [41]:
class PortfolioOptimizer:
    """Optimización de carteras usando Markowitz"""
    
    def __init__(self, returns_data, risk_free_rate=0.02):
        self.returns = returns_data.dropna()
        self.assets = self.returns.columns.tolist()
        self.n_assets = len(self.assets)
        self.rf_rate = risk_free_rate / 252
        
    def negative_sharpe(self, weights):
        """Función objetivo para maximizar Sharpe"""
        portfolio_return = np.sum(self.returns.mean() * weights) * 252
        portfolio_std = np.sqrt(np.dot(weights.T, np.dot(self.returns.cov() * 252, weights)))
        sharpe = (portfolio_return - self.rf_rate * 252) / portfolio_std
        return -sharpe
    
    def optimize_sharpe(self):
        """Optimiza para máximo Sharpe Ratio"""
        constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
        bounds = tuple((0, 1) for _ in range(self.n_assets))
        initial = np.array([1/self.n_assets] * self.n_assets)
        
        try:
            result = minimize(self.negative_sharpe, initial, method='SLSQP', 
                            bounds=bounds, constraints=constraints)
            
            if result.success:
                weights = dict(zip(self.assets, result.x))
                portfolio_return = np.sum(self.returns.mean() * result.x) * 252
                portfolio_std = np.sqrt(np.dot(result.x.T, np.dot(self.returns.cov() * 252, result.x)))
                sharpe = (portfolio_return - self.rf_rate * 252) / portfolio_std
                
                return {
                    'tipo': 'Máximo Sharpe',
                    'weights': weights,
                    'rendimiento': portfolio_return,
                    'volatilidad': portfolio_std,
                    'sharpe_ratio': sharpe
                }
        except:
            pass
        return None
    
    def optimize_min_vol(self):
        """Optimiza para mínima volatilidad"""
        def portfolio_vol(weights):
            return np.sqrt(np.dot(weights.T, np.dot(self.returns.cov() * 252, weights)))
        
        constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
        bounds = tuple((0, 1) for _ in range(self.n_assets))
        initial = np.array([1/self.n_assets] * self.n_assets)
        
        try:
            result = minimize(portfolio_vol, initial, method='SLSQP',
                            bounds=bounds, constraints=constraints)
            
            if result.success:
                weights = dict(zip(self.assets, result.x))
                portfolio_return = np.sum(self.returns.mean() * result.x) * 252
                portfolio_std = result.fun
                sharpe = (portfolio_return - self.rf_rate * 252) / portfolio_std
                
                return {
                    'tipo': 'Mínimo Riesgo',
                    'weights': weights,
                    'rendimiento': portfolio_return,
                    'volatilidad': portfolio_std,
                    'sharpe_ratio': sharpe
                }
        except:
            pass
        return None

def calcular_retornos_cartera(df, weights_dict):
    """Calcula retornos históricos de una cartera"""
    portfolio_returns = []
    
    for i in range(1, len(df)):
        day_return = 0
        valid_weights_sum = 0
        
        for activo, peso in weights_dict.items():
            if activo in df.columns:
                if pd.notna(df.iloc[i][activo]) and pd.notna(df.iloc[i-1][activo]):
                    ret_activo = (df.iloc[i][activo] / df.iloc[i-1][activo]) - 1
                    day_return += peso * ret_activo
                    valid_weights_sum += peso
        
        if valid_weights_sum > 0:
            day_return = day_return / valid_weights_sum
        
        portfolio_returns.append(day_return)
    
    return portfolio_returns

# OPTIMIZACIÓN DE CARTERAS
print(f"📊 Activos disponibles: {len(activos_verdaderos)} activos verdaderos")
print(f"📊 Todos los activos: {activos_verdaderos}")

if not returns_data.empty and len(returns_data.columns) >= 2:
    
    # Configurar tasa libre de riesgo
    tasa_rf = 0.02  # Default
    if 'df_data_completo' in globals() and 'Tasa_PlazoFijo_Diaria' in df_data_completo.columns:
        tasa_diaria = df_data_completo['Tasa_PlazoFijo_Diaria'].dropna().mean()
        tasa_rf = ((1 + tasa_diaria) ** 252) - 1
    
    # Optimizar
    optimizer = PortfolioOptimizer(returns_data, risk_free_rate=tasa_rf)
    cartera_opt_sharpe = optimizer.optimize_sharpe()
    cartera_opt_vol = optimizer.optimize_min_vol()
    
    # RESULTADOS DE OPTIMIZACIÓN
    carteras_data = []
    
    if cartera_opt_sharpe:
        carteras_data.append({
            'Cartera': 'Máximo Sharpe (Optimizada)',
            'Rendimiento_Anual_%': f"{cartera_opt_sharpe['rendimiento']*100:.2f}",
            'Volatilidad_%': f"{cartera_opt_sharpe['volatilidad']*100:.2f}",
            'Sharpe_Ratio': f"{cartera_opt_sharpe['sharpe_ratio']:.3f}"
        })
    
    if cartera_opt_vol:
        carteras_data.append({
            'Cartera': 'Mínimo Riesgo (Optimizada)',
            'Rendimiento_Anual_%': f"{cartera_opt_vol['rendimiento']*100:.2f}",
            'Volatilidad_%': f"{cartera_opt_vol['volatilidad']*100:.2f}",
            'Sharpe_Ratio': f"{cartera_opt_vol['sharpe_ratio']:.3f}"
        })
    
    # AGREGAR CARTERA REAL SI EXISTE
    cartera_real_weights = None
    if 'cartera_real' in globals() and cartera_real:
        # La cartera real puede tener estructura diferente, extraer solo los pesos
        if isinstance(cartera_real, dict):
            if 'weights' in cartera_real:
                cartera_real_weights = cartera_real['weights']
            else:
                # Asumir que cartera_real ya es el diccionario de pesos
                # Filtrar solo valores numéricos
                cartera_real_weights = {k: v for k, v in cartera_real.items() 
                                      if isinstance(v, (int, float)) and not isinstance(v, dict)}
        
        if cartera_real_weights:
            # Calcular métricas de la cartera real
            portfolio_returns_real = calcular_retornos_cartera(df_data_completo, cartera_real_weights)
            if len(portfolio_returns_real) > 30:
                ret_real = pd.Series(portfolio_returns_real)
                rend_anual_real = ret_real.mean() * 252
                vol_real = ret_real.std() * np.sqrt(252)
                sharpe_real = (rend_anual_real - tasa_rf) / vol_real if vol_real > 0 else 0
                
                carteras_data.append({
                    'Cartera': 'Cartera Real (Actual)',
                    'Rendimiento_Anual_%': f"{rend_anual_real*100:.2f}",
                    'Volatilidad_%': f"{vol_real*100:.2f}",
                    'Sharpe_Ratio': f"{sharpe_real:.3f}"
                })
    
    # CARTERA EQUAL WEIGHT
    equal_weights = {activo: 1/len(activos_verdaderos) for activo in activos_verdaderos}
    portfolio_returns_eq = calcular_retornos_cartera(df_data_completo, equal_weights)
    if len(portfolio_returns_eq) > 30:
        ret_eq = pd.Series(portfolio_returns_eq)
        rend_anual_eq = ret_eq.mean() * 252
        vol_eq = ret_eq.std() * np.sqrt(252)
        sharpe_eq = (rend_anual_eq - tasa_rf) / vol_eq
        
        carteras_data.append({
            'Cartera': 'Equal Weight (Benchmark)',
            'Rendimiento_Anual_%': f"{rend_anual_eq*100:.2f}",
            'Volatilidad_%': f"{vol_eq*100:.2f}",
            'Sharpe_Ratio': f"{sharpe_eq:.3f}"
        })
    
    if carteras_data:
        print("🎯 CARTERAS OPTIMIZADAS")
        print("="*50)
        df_carteras_opt = pd.DataFrame(carteras_data)
        display(df_carteras_opt)
        
        # COMPOSICIONES (solo pesos > 1%)
        carteras_para_mostrar = []
        if cartera_opt_sharpe: carteras_para_mostrar.append(cartera_opt_sharpe)
        if cartera_opt_vol: carteras_para_mostrar.append(cartera_opt_vol)
        if cartera_real_weights: carteras_para_mostrar.append({'tipo': 'Cartera Real', 'weights': cartera_real_weights})
        
        for cartera in carteras_para_mostrar:
            print(f"\n📊 Composición {cartera['tipo']}:")
            # Verificar que weights sea un diccionario válido
            if 'weights' in cartera and isinstance(cartera['weights'], dict):
                pesos_significativos = {k: v for k, v in cartera['weights'].items() 
                                      if isinstance(v, (int, float)) and v > 0.01}
                df_pesos = pd.DataFrame([
                    {'Activo': activo, 'Peso_%': f"{peso*100:.1f}"} 
                    for activo, peso in sorted(pesos_significativos.items(), key=lambda x: x[1], reverse=True)
                ])
                display(df_pesos)
        
        # MÉTRICAS AVANZADAS DE LAS CARTERAS OPTIMIZADAS
        calculator = RiskCalculator()
        calculator.set_dynamic_risk_free_rate(df_data_completo)
        
        metricas_carteras = []
        
        # Analizar cartera Sharpe
        if cartera_opt_sharpe:
            portfolio_returns = calcular_retornos_cartera(df_data_completo, cartera_opt_sharpe['weights'])
            if len(portfolio_returns) > 30:
                metrics = calculator.calculate_metrics(pd.Series(portfolio_returns), df_data_completo, "Sharpe_Opt")
                if metrics:
                    metrics['Cartera'] = 'Máximo Sharpe (Optimizada)'
                    metricas_carteras.append(metrics)
        
        # Analizar cartera mínimo riesgo
        if cartera_opt_vol:
            portfolio_returns = calcular_retornos_cartera(df_data_completo, cartera_opt_vol['weights'])
            if len(portfolio_returns) > 30:
                metrics = calculator.calculate_metrics(pd.Series(portfolio_returns), df_data_completo, "MinVol_Opt")
                if metrics:
                    metrics['Cartera'] = 'Mínimo Riesgo (Optimizada)'
                    metricas_carteras.append(metrics)
        
        # Analizar cartera real si existe
        if cartera_real_weights:
            portfolio_returns = calcular_retornos_cartera(df_data_completo, cartera_real_weights)
            if len(portfolio_returns) > 30:
                metrics = calculator.calculate_metrics(pd.Series(portfolio_returns), df_data_completo, "Cartera_Real")
                if metrics:
                    metrics['Cartera'] = 'Cartera Real (Actual)'
                    metricas_carteras.append(metrics)
        
        # Cartera Equal Weight como benchmark
        equal_weights = {activo: 1/len(activos_verdaderos) for activo in activos_verdaderos}
        portfolio_returns = calcular_retornos_cartera(df_data_completo, equal_weights)
        if len(portfolio_returns) > 30:
            metrics = calculator.calculate_metrics(pd.Series(portfolio_returns), df_data_completo, "Equal_Weight")
            if metrics:
                metrics['Cartera'] = 'Equal Weight (Benchmark)'
                metricas_carteras.append(metrics)
        
        if metricas_carteras:
            df_metricas_carteras = pd.DataFrame(metricas_carteras)
            
            print(f"\n📈 MÉTRICAS DE RIESGO Y PERFORMANCE")
            print("="*50)
            
            # Formatear para display
            cols_display = ['Cartera', 'rendimiento_anual', 'volatilidad', 'sharpe_ratio', 'max_drawdown']
            df_show = df_metricas_carteras[cols_display].copy()
            df_show['rendimiento_anual'] = (df_show['rendimiento_anual'] * 100).round(2)
            df_show['volatilidad'] = (df_show['volatilidad'] * 100).round(2)
            df_show['sharpe_ratio'] = df_show['sharpe_ratio'].round(3)
            df_show['max_drawdown'] = (df_show['max_drawdown'] * 100).round(2)
            df_show.columns = ['Cartera', 'Rendimiento_Anual_%', 'Volatilidad_%', 'Sharpe_Ratio', 'Max_Drawdown_%']
            display(df_show)
            
            # Métricas avanzadas
            if any(col in df_metricas_carteras.columns for col in ['alpha', 'beta', 'kelly_criterion']):
                print(f"\n🔸 MÉTRICAS AVANZADAS")
                cols_advanced = ['Cartera', 'alpha', 'beta', 'kelly_criterion', 'calmar_ratio']
                cols_available = [col for col in cols_advanced if col in df_metricas_carteras.columns]
                
                if len(cols_available) > 1:
                    df_advanced = df_metricas_carteras[cols_available].copy()
                    for col in ['alpha', 'beta', 'kelly_criterion', 'calmar_ratio']:
                        if col in df_advanced.columns:
                            df_advanced[col] = df_advanced[col].round(4)
                    display(df_advanced)
        
    else:
        print("❌ No se pudieron optimizar carteras")
        cartera_opt_sharpe = cartera_opt_vol = None

else:
    print("❌ Datos insuficientes para optimización (mínimo 2 activos)")
    cartera_opt_sharpe = cartera_opt_vol = None

📊 Activos disponibles: 7 activos verdaderos
📊 Todos los activos: ['AAPL', 'SPY', 'EWZ', 'CEPU', 'BHIL', 'METRO', 'IBM']
🎯 CARTERAS OPTIMIZADAS
🎯 CARTERAS OPTIMIZADAS


,Cartera,Rendimiento_Anual_%,Volatilidad_%,Sharpe_Ratio
0,Máximo Sharpe (Optimizada),63.73,28.37,1.350
1,Mínimo Riesgo (Optimizada),33.68,21.92,0.376
2,Cartera Real (Actual),36.76,26.05,0.434
3,Equal Weight (Benchmark),-6.02,31.01,-1.014



📊 Composición Máximo Sharpe:


,Activo,Peso_%
0,EWZ,100.0



📊 Composición Mínimo Riesgo:


...

,Activo,Peso_%
0,SPY,52.9
1,EWZ,24.0
2,AAPL,18.9
3,METRO,1.8
4,CEPU,1.7


✅ Tasa libre de riesgo: 32.85% anual

📈 MÉTRICAS DE RIESGO Y PERFORMANCE

📈 MÉTRICAS DE RIESGO Y PERFORMANCE


,Cartera,Rendimiento_Anual_%,Volatilidad_%,Sharpe_Ratio,Max_Drawdown_%
0,Máximo Sharpe (Optimizada),63.7300,28.3700,1.0890,-17.8900
1,Mínimo Riesgo (Optimizada),33.6800,21.9200,0.0380,-15.2800
2,Cartera Real (Actual),36.7600,26.0500,0.1500,-21.1200
3,Equal Weight (Benchmark),-6.0200,31.0100,-1.2530,-21.8000



🔸 MÉTRICAS AVANZADAS


,Cartera,alpha,beta,kelly_criterion,calmar_ratio
0,Máximo Sharpe (Optimizada),0.3011,0.8696,0.1809,3.5616
1,Mínimo Riesgo (Optimizada),0.0582,0.7207,0.1350,2.2044
2,Cartera Real (Actual),-0.0155,0.9911,0.1319,1.7400
3,Equal Weight (Benchmark),-0.3528,0.7570,0.0000,-0.2760


## 5. Backtesting y Validación de Estrategias

Validación histórica de las carteras optimizadas.

In [42]:
# =============================================================================
# 🔄 BACKTESTING CON BENCHMARKS ESPECÍFICOS PARA TU CARTERA
# =============================================================================

def backtest_estrategia(df, weights_dict, nombre):
    """Ejecuta backtesting de una estrategia"""
    
    # Calcular retornos de cartera
    portfolio_returns = []
    fechas = []
    
    for i in range(1, len(df)):
        if 'Fecha' in df.columns:
            fecha = df.iloc[i]['Fecha']
        else:
            fecha = df.index[i] if hasattr(df.index[i], 'date') else f"Día {i}"
        
        port_return = 0
        valid_weights_sum = 0
        
        for activo, peso in weights_dict.items():
            if activo in df.columns:
                if pd.notna(df.iloc[i][activo]) and pd.notna(df.iloc[i-1][activo]):
                    ret_activo = (df.iloc[i][activo] / df.iloc[i-1][activo]) - 1
                    port_return += peso * ret_activo
                    valid_weights_sum += peso
        
        # Normalizar por pesos válidos
        if valid_weights_sum > 0:
            port_return = port_return / valid_weights_sum
            
        portfolio_returns.append(port_return)
        fechas.append(fecha)
    
    # Calcular métricas
    port_returns = pd.Series(portfolio_returns)
    valor_acumulado = (1 + port_returns).cumprod() * 100
    
    ret_anual = port_returns.mean() * 252
    vol_anual = port_returns.std() * np.sqrt(252)
    
    # Usar tasa libre de riesgo real
    rf_rate = tasa_rf if 'tasa_rf' in globals() else 0.02
    sharpe = (ret_anual - rf_rate) / vol_anual if vol_anual > 0 else 0
    
    # Maximum Drawdown
    running_max = valor_acumulado.cummax()
    drawdown = (valor_acumulado - running_max) / running_max
    max_dd = drawdown.min()
    
    return {
        'nombre': nombre,
        'fechas': fechas,
        'valores': valor_acumulado.tolist(),
        'rendimiento_anual': ret_anual,
        'volatilidad_anual': vol_anual,
        'sharpe_ratio': sharpe,
        'max_drawdown': max_dd
    }

print("🏆 BACKTESTING CON BENCHMARKS ESPECÍFICOS PARA TU CARTERA")
print("="*70)

estrategias = []

# 1. CARTERA REAL (TU CARTERA ACTUAL)
if 'cartera_real_weights' in globals() and cartera_real_weights:
    print("✅ Analizando tu cartera real...")
    bt_real = backtest_estrategia(df_data_completo, cartera_real_weights, 'Tu Cartera Real')
    estrategias.append(bt_real)

# 2. CARTERA OPTIMIZADA MAXIMO SHARPE (si existe)
if 'cartera_opt_sharpe' in globals() and cartera_opt_sharpe:
    print("✅ Analizando cartera óptima Sharpe...")
    bt_opt_sharpe = backtest_estrategia(df_data_completo, cartera_opt_sharpe['weights'], 'Cartera Óptima Sharpe')
    estrategias.append(bt_opt_sharpe)

# 3. CARTERA OPTIMIZADA MINIMO RIESGO (si existe)
if 'cartera_opt_vol' in globals() and cartera_opt_vol:
    print("✅ Analizando cartera óptima mínimo riesgo...")
    bt_opt_vol = backtest_estrategia(df_data_completo, cartera_opt_vol['weights'], 'Cartera Óptima MinVol')
    estrategias.append(bt_opt_vol)

# =============================================================================
# 🎯 BENCHMARKS ESPECÍFICOS BASADOS EN TU CARTERA (NO RANDOM!)
# =============================================================================

if 'cartera_real_weights' in globals() and cartera_real_weights:
    # Obtener SOLO los activos que TÚ elegiste
    activos_en_cartera = [activo for activo, peso in cartera_real_weights.items() if peso > 0.001]
    print(f"\n🎯 Creando benchmarks específicos para TUS {len(activos_en_cartera)} activos:")
    print(f"   Activos seleccionados: {activos_en_cartera}")
    
    # BENCHMARK 1: Equal Weight SOLO de tus activos elegidos
    print("📊 Benchmark 1: Equal Weight (solo tus activos)")
    equal_weight_tu_cartera = {activo: 1/len(activos_en_cartera) for activo in activos_en_cartera}
    bt_equal_tuyo = backtest_estrategia(df_data_completo, equal_weight_tu_cartera, 'Equal Weight (Tus Activos)')
    estrategias.append(bt_equal_tuyo)
    
    # BENCHMARK 2: Balanceado Argentina vs Internacional
    argentinos = []
    internacionales = []
    
    for activo in activos_en_cartera:
        activo_upper = activo.upper()
        if activo_upper in ['GGAL', 'PAMP', 'TXAR', 'BBAR', 'YPF', 'ALUA', 'MIRG', 'TECO2']:
            argentinos.append(activo)
        else:
            internacionales.append(activo)
    
    if len(argentinos) > 0 and len(internacionales) > 0:
        print(f"📊 Benchmark 2: Argentina 60% ({len(argentinos)} activos) / Internacional 40% ({len(internacionales)} activos)")
        benchmark_60_40 = {}
        # 60% Argentina, 40% Internacional
        peso_arg = 0.6 / len(argentinos)
        peso_int = 0.4 / len(internacionales)
        
        for activo in argentinos:
            benchmark_60_40[activo] = peso_arg
        for activo in internacionales:
            benchmark_60_40[activo] = peso_int
            
        bt_60_40 = backtest_estrategia(df_data_completo, benchmark_60_40, 'Benchmark Argentina 60/40')
        estrategias.append(bt_60_40)
        
        print(f"   • Argentinos: {argentinos}")
        print(f"   • Internacionales: {internacionales}")
    
    # BENCHMARK 3: Balanceado por capitalización simulada
    print("📊 Benchmark 3: Cap-Weighted (simulado por tamaño)")
    benchmark_cap_weighted = {}
    cap_weights = {}
    
    for activo in activos_en_cartera:
        activo_upper = activo.upper()
        # Asignar pesos por capitalización aproximada
        if activo_upper in ['AAPL', 'MSFT', 'GOOGL', 'AMZN']:
            cap_weights[activo] = 4.0  # Mega caps
        elif activo_upper in ['SPY', 'QQQ', 'VTI']:
            cap_weights[activo] = 3.5  # ETFs grandes
        elif activo_upper in ['GGAL', 'PAMP', 'YPF']:
            cap_weights[activo] = 2.5  # Large caps argentinas
        elif activo_upper in ['BBAR', 'TXAR']:
            cap_weights[activo] = 2.0  # Mid caps argentinas
        else:
            cap_weights[activo] = 1.0  # Resto
    
    # Normalizar
    total_cap_weight = sum(cap_weights.values())
    for activo, peso in cap_weights.items():
        benchmark_cap_weighted[activo] = peso / total_cap_weight
    
    bt_cap = backtest_estrategia(df_data_completo, benchmark_cap_weighted, 'Benchmark Cap-Weighted')
    estrategias.append(bt_cap)
    
    # Mostrar pesos del benchmark cap-weighted
    print("   • Pesos por capitalización:")
    for activo, peso in sorted(benchmark_cap_weighted.items(), key=lambda x: x[1], reverse=True):
        print(f"     - {activo}: {peso*100:.1f}%")

# =============================================================================
# 📊 MOSTRAR RESULTADOS COMPARATIVOS
# =============================================================================

if estrategias:
    print(f"\n📊 RESULTADOS BACKTESTING CON BENCHMARKS ESPECÍFICOS:")
    print("="*70)
    
    resultados_df = pd.DataFrame([
        {
            'Estrategia': est['nombre'],
            'Rendimiento_Anual_%': f"{est['rendimiento_anual']*100:.2f}",
            'Volatilidad_Anual_%': f"{est['volatilidad_anual']*100:.2f}",
            'Sharpe_Ratio': f"{est['sharpe_ratio']:.3f}",
            'Max_Drawdown_%': f"{est['max_drawdown']*100:.2f}",
            'Valor_Final': f"{est['valores'][-1]:.1f}",
            'Tipo': '🎯 TU CARTERA' if 'Tu Cartera Real' in est['nombre'] 
                   else '🏆 OPTIMIZADA' if 'Óptima' in est['nombre']
                   else '📊 BENCHMARK'
        }
        for est in estrategias
    ])
    
    # Ordenar por Sharpe Ratio
    resultados_df = resultados_df.sort_values('Sharpe_Ratio', ascending=False)
    
    display(resultados_df)
    
    # =============================================================================
    # 📈 GRÁFICO COMPARATIVO MEJORADO
    # =============================================================================
    
    fig = go.Figure()
    
    # Colores específicos
    color_map = {
        'Tu Cartera Real': '#e74c3c',  # Rojo brillante para destacar
        'Cartera Óptima Sharpe': '#2ecc71',  # Verde
        'Cartera Óptima MinVol': '#3498db',   # Azul
        'Equal Weight (Tus Activos)': '#f39c12',  # Naranja
        'Benchmark Argentina 60/40': '#9b59b6',   # Púrpura
        'Benchmark Cap-Weighted': '#1abc9c'       # Turquesa
    }
    
    for est in estrategias:
        color = color_map.get(est['nombre'], '#34495e')
        
        # Tu cartera con línea más gruesa
        if 'Tu Cartera Real' in est['nombre']:
            width = 4.0
            dash = 'solid'
        elif 'Benchmark' in est['nombre']:
            width = 2.0
            dash = 'dash'
        else:
            width = 2.5
            dash = 'solid'
        
        fig.add_trace(go.Scatter(
            x=est['fechas'],
            y=est['valores'],
            mode='lines',
            name=est['nombre'],
            line=dict(width=width, color=color, dash=dash),
            hovertemplate=f"{est['nombre']}<br>Valor: %{{y:.1f}}<br>Sharpe: {est['sharpe_ratio']:.3f}<extra></extra>"
        ))
    
    fig.update_layout(
        title='🏆 Tu Cartera vs Benchmarks Específicos (NO Equal Weight Random)',
        xaxis_title='Período',
        yaxis_title='Valor de Cartera (Base 100)',
        template='plotly_white',
        height=700,
        hovermode='x unified',
        font=dict(size=12),
        title_font_size=16,
        legend=dict(
            orientation="v",
            yanchor="top",
            y=0.99,
            xanchor="left", 
            x=1.02
        )
    )
    
    # Línea de referencia
    fig.add_hline(y=100, line_dash="dot", line_color="gray", 
                  annotation_text="Valor Inicial", annotation_position="bottom right")
    
    fig.show()
    
    # Guardar gráfico
    try:
        fig.write_image("backtesting_benchmarks_especificos.png", width=1400, height=700, scale=2)
        print("📁 Gráfico guardado como 'backtesting_benchmarks_especificos.png'")
    except:
        print("⚠️ No se pudo guardar el gráfico")
    
    # =============================================================================
    # 🎯 ANÁLISIS DETALLADO DE TU CARTERA
    # =============================================================================
    
    cartera_real_result = next((est for est in estrategias if 'Tu Cartera Real' in est['nombre']), None)
    if cartera_real_result:
        print(f"\n🎯 ANÁLISIS DETALLADO DE TU CARTERA VS BENCHMARKS ESPECÍFICOS:")
        print("="*65)
        
        # Tu cartera
        print(f"📈 TU CARTERA REAL:")
        print(f"   • Rendimiento Anualizado:    {cartera_real_result['rendimiento_anual']*100:>7.2f}%")
        print(f"   • Volatilidad Anualizada:    {cartera_real_result['volatilidad_anual']*100:>7.2f}%")
        print(f"   • Sharpe Ratio:              {cartera_real_result['sharpe_ratio']:>7.3f}")
        print(f"   • Máximo Drawdown:           {cartera_real_result['max_drawdown']*100:>7.2f}%")
        print(f"   • Valor Final (Base 100):    {cartera_real_result['valores'][-1]:>7.1f}")
        
        # Ranking
        ranking = resultados_df.reset_index()
        posicion_tu_cartera = ranking[ranking['Estrategia'] == 'Tu Cartera Real'].index[0] + 1
        total_estrategias = len(ranking)
        
        print(f"\n🏆 RANKING POR SHARPE RATIO:")
        print(f"   • Tu posición: {posicion_tu_cartera}° de {total_estrategias}")
        
        if posicion_tu_cartera == 1:
            print(f"   🥇 ¡TU CARTERA ES LA MEJOR!")
        elif posicion_tu_cartera <= total_estrategias * 0.3:
            print(f"   ✅ Tu cartera está en el TOP 30%")
        elif posicion_tu_cartera <= total_estrategias * 0.5:
            print(f"   📊 Tu cartera está en la mitad superior")
        else:
            print(f"   ⚠️ Tu cartera tiene potencial de mejora")
        
        # Comparación con mejor benchmark
        mejor_benchmark = None
        for _, row in ranking.iterrows():
            if row['Tipo'] == '📊 BENCHMARK':
                mejor_benchmark = next((est for est in estrategias if est['nombre'] == row['Estrategia']), None)
                break
        
        if mejor_benchmark:
            print(f"\n📊 VS MEJOR BENCHMARK ({mejor_benchmark['nombre']}):")
            diff_rend = cartera_real_result['rendimiento_anual'] - mejor_benchmark['rendimiento_anual']
            diff_sharpe = cartera_real_result['sharpe_ratio'] - mejor_benchmark['sharpe_ratio']
            diff_dd = cartera_real_result['max_drawdown'] - mejor_benchmark['max_drawdown']
            
            print(f"   • Diferencia Rendimiento:    {diff_rend*100:>+7.2f}%")
            print(f"   • Diferencia Sharpe:         {diff_sharpe:>+7.3f}")
            print(f"   • Diferencia Drawdown:       {diff_dd*100:>+7.2f}%")
            
            if diff_sharpe > 0:
                print(f"   ✅ Tu cartera supera al mejor benchmark")
            else:
                print(f"   📊 El benchmark tiene mejor ratio riesgo/retorno")
    
    print(f"\n🎯 RESUMEN:")
    print(f"✅ Benchmarks creados usando EXACTAMENTE TUS activos elegidos")
    print(f"📊 NO se usó 'Equal Weight' de todos los activos disponibles")
    print(f"🏆 Comparación justa: mismos activos, diferentes asignaciones")
    
else:
    print("❌ No se encontraron carteras para backtesting")

🏆 BACKTESTING CON BENCHMARKS ESPECÍFICOS PARA TU CARTERA
✅ Analizando tu cartera real...
✅ Analizando cartera óptima Sharpe...
✅ Analizando cartera óptima Sharpe...
✅ Analizando cartera óptima mínimo riesgo...
✅ Analizando cartera óptima mínimo riesgo...

🎯 Creando benchmarks específicos para TUS 7 activos:
   Activos seleccionados: ['AAPL', 'SPY', 'EWZ', 'CEPU', 'BHIL', 'METRO', 'IBM']
📊 Benchmark 1: Equal Weight (solo tus activos)

🎯 Creando benchmarks específicos para TUS 7 activos:
   Activos seleccionados: ['AAPL', 'SPY', 'EWZ', 'CEPU', 'BHIL', 'METRO', 'IBM']
📊 Benchmark 1: Equal Weight (solo tus activos)
📊 Benchmark 3: Cap-Weighted (simulado por tamaño)
📊 Benchmark 3: Cap-Weighted (simulado por tamaño)
   • Pesos por capitalización:
     - AAPL: 32.0%
     - SPY: 28.0%
     - EWZ: 8.0%
     - CEPU: 8.0%
     - BHIL: 8.0%
     - METRO: 8.0%
     - IBM: 8.0%

📊 RESULTADOS BACKTESTING CON BENCHMARKS ESPECÍFICOS:
   • Pesos por capitalización:
     - AAPL: 32.0%
     - SPY: 28.0%
  

,Estrategia,Rendimiento_Anual_%,Volatilidad_Anual_%,Sharpe_Ratio,Max_Drawdown_%,Valor_Final,Tipo
1,Cartera Óptima Sharpe,63.73,28.37,1.350,-17.89,149.9,🏆 OPTIMIZADA
0,Tu Cartera Real,36.76,26.05,0.434,-21.12,125.4,🎯 TU CARTERA
2,Cartera Óptima MinVol,33.68,21.92,0.376,-15.28,123.6,🏆 OPTIMIZADA
3,Equal Weight (Tus Activos),-6.02,31.01,-1.014,-21.80,92.9,📊 BENCHMARK
4,Benchmark Cap-Weighted,8.93,28.15,-0.587,-21.64,103.4,📊 BENCHMARK


📁 Gráfico guardado como 'backtesting_benchmarks_especificos.png'

🎯 ANÁLISIS DETALLADO DE TU CARTERA VS BENCHMARKS ESPECÍFICOS:
📈 TU CARTERA REAL:
   • Rendimiento Anualizado:      36.76%
   • Volatilidad Anualizada:      26.05%
   • Sharpe Ratio:                0.434
   • Máximo Drawdown:            -21.12%
   • Valor Final (Base 100):      125.4

🏆 RANKING POR SHARPE RATIO:
   • Tu posición: 2° de 5
   📊 Tu cartera está en la mitad superior

📊 VS MEJOR BENCHMARK (Equal Weight (Tus Activos)):
   • Diferencia Rendimiento:     +42.77%
   • Diferencia Sharpe:          +1.449
   • Diferencia Drawdown:         +0.67%
   ✅ Tu cartera supera al mejor benchmark

🎯 RESUMEN:
✅ Benchmarks creados usando EXACTAMENTE TUS activos elegidos
📊 NO se usó 'Equal Weight' de todos los activos disponibles
🏆 Comparación justa: mismos activos, diferentes asignaciones


## 6. Análisis de Escenarios y Stress Testing

Simulaciones Monte Carlo y análisis de escenarios adversos.

In [43]:
# =============================================================================
# 📦 INSTALACIÓN DE LIBRERÍAS PARA MODELADO ARCH/GARCH
# =============================================================================

try:
    from arch import arch_model
    print("✅ Librería ARCH ya instalada")
except ImportError:
    print("📦 Instalando librería ARCH...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "arch"])
    from arch import arch_model
    print("✅ Librería ARCH instalada correctamente")

# Importaciones adicionales para GARCH
import warnings
warnings.filterwarnings('ignore')

print("📊 Librerías ARCH/GARCH listas para usar")

✅ Librería ARCH ya instalada
📊 Librerías ARCH/GARCH listas para usar


In [44]:
# =============================================================================
# 🔮 SIMULADOR MONTE CARLO CON MODELOS ARCH/GARCH PARA ARGENTINA
# =============================================================================

class MonteCarloArgentinaARCH:
    """
    Simulador Monte Carlo con modelos ARCH/GARCH para carteras en el contexto argentino.
    Modelado avanzado de volatilidad condicional.
    """
    
    def __init__(self, risk_free_rate=0.04):
        """Inicializa con parámetros ajustados para Argentina"""
        self.rf_rate = risk_free_rate
        self.peso_usd_volatility = 0.35
        np.random.seed(42)
        
        # Factores de ajuste para Argentina con ARCH
        self.asset_adjustments = {
            'argentina_stocks': {'vol_factor': 1.2, 'crisis_sensitivity': 1.5, 'arch_order': (1,1)},
            'international_etf': {'vol_factor': 1.0, 'crisis_sensitivity': 1.0, 'arch_order': (1,1)},
            'us_stocks': {'vol_factor': 1.0, 'crisis_sensitivity': 0.9, 'arch_order': (1,1)},
            'commodities': {'vol_factor': 0.8, 'crisis_sensitivity': -0.2, 'arch_order': (1,0)},
            'bonds_local': {'vol_factor': 1.1, 'crisis_sensitivity': 1.3, 'arch_order': (2,1)}
        }
        
        # Escenarios específicos para Argentina
        self.scenarios_argentina = {
            'Crisis Cambiaria': {'shock': -0.50, 'duration': 90, 'prob': 0.15},
            'Devaluación Fuerte': {'shock': -0.35, 'duration': 60, 'prob': 0.20},
            'Crisis Política': {'shock': -0.40, 'duration': 120, 'prob': 0.10},
            'Recesión Local': {'shock': -0.25, 'duration': 180, 'prob': 0.25},
            'Hiperinflación': {'shock': -0.30, 'duration': 252, 'prob': 0.05},
            'Default Soberano': {'shock': -0.60, 'duration': 365, 'prob': 0.08},
            'Crisis Global + Local': {'shock': -0.70, 'duration': 200, 'prob': 0.03}
        }
    
    def classify_asset_argentina(self, asset_name):
        """Clasifica activos según el contexto argentino"""
        asset_upper = asset_name.upper()
        
        if asset_upper in ['GGAL', 'PAMP', 'TXAR', 'BBAR', 'YPF', 'ALUA', 'MIRG', 'TECO2']:
            return 'argentina_stocks'
        elif asset_upper in ['EWZ', 'SPY', 'QQQ', 'IWM', 'VTI', 'ARGT']:
            return 'international_etf'
        elif asset_upper in ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'IBM', 'META']:
            return 'us_stocks'
        elif asset_upper in ['GOLD', 'GLD', 'SLV', 'DJP']:
            return 'commodities'
        else:
            return 'us_stocks'
    
    def fit_garch_model(self, returns_series, asset_type='international_etf'):
        """
        Ajusta modelo GARCH/ARCH a una serie de retornos
        """
        try:
            # Limpiar datos
            returns_clean = returns_series.dropna()
            returns_clean = returns_clean.replace([np.inf, -np.inf], np.nan).dropna()
            
            if len(returns_clean) < 50:
                print(f"   ⚠️ Pocos datos para GARCH, usando parámetros por defecto")
                return None
            
            # Convertir a porcentajes para mejor convergencia
            returns_pct = returns_clean * 100
            
            # Obtener orden ARCH del tipo de activo
            p, q = self.asset_adjustments.get(asset_type, {}).get('arch_order', (1,1))
            
            # Definir modelo GARCH(p,q)
            if q == 0:
                # Modelo ARCH puro
                model = arch_model(returns_pct, vol='ARCH', p=p, rescale=False)
            else:
                # Modelo GARCH
                model = arch_model(returns_pct, vol='GARCH', p=p, q=q, rescale=False)
            
            # Ajustar modelo
            fitted_model = model.fit(disp='off', show_warning=False)
            
            print(f"   ✅ Modelo GARCH({p},{q}) ajustado exitosamente")
            print(f"   📊 AIC: {fitted_model.aic:.2f}, Log-Likelihood: {fitted_model.loglikelihood:.2f}")
            
            return fitted_model
            
        except Exception as e:
            print(f"   ❌ Error en GARCH: {str(e)[:50]}...")
            return None
    
    def calculate_portfolio_returns_with_garch(self, weights_dict, returns_data):
        """Calcula retornos de cartera y ajusta modelos GARCH por activo"""
        
        print("📊 Calculando retornos y ajustando modelos GARCH por activo...")
        
        # Calcular retornos de cartera día a día
        portfolio_returns = []
        asset_garch_models = {}
        
        for i in range(1, len(returns_data)):
            day_return = 0
            valid_weights_sum = 0
            
            for asset, weight in weights_dict.items():
                if asset in returns_data.columns and weight > 0.001:
                    current_price = returns_data.iloc[i][asset]
                    previous_price = returns_data.iloc[i-1][asset]
                    
                    if pd.notna(current_price) and pd.notna(previous_price) and previous_price != 0:
                        # RETORNO REAL SIN AJUSTES ARTIFICIALES
                        asset_return = (current_price / previous_price) - 1
                        day_return += weight * asset_return
                        valid_weights_sum += weight
            
            if valid_weights_sum > 0:
                day_return = day_return / valid_weights_sum
                portfolio_returns.append(day_return)
        
        portfolio_returns = np.array(portfolio_returns)
        portfolio_returns = portfolio_returns[~np.isnan(portfolio_returns)]
        
        # Ajustar modelo GARCH al retorno de cartera
        print(f"\n🔮 AJUSTE DE MODELO GARCH PARA CARTERA COMPLETA:")
        portfolio_garch = self.fit_garch_model(
            pd.Series(portfolio_returns), 
            'international_etf'  # Promedio ponderado
        )
        
        # También ajustar GARCH por activo individual (para análisis)
        print(f"\n📈 AJUSTE DE MODELOS GARCH POR ACTIVO:")
        for asset, weight in sorted(weights_dict.items(), key=lambda x: x[1], reverse=True):
            if weight > 0.05 and asset in returns_data.columns:  # Solo activos >5%
                asset_returns = returns_data[asset].pct_change().dropna()
                if len(asset_returns) > 50:
                    asset_type = self.classify_asset_argentina(asset)
                    print(f"   🔸 {asset} ({asset_type}):")
                    garch_model = self.fit_garch_model(asset_returns, asset_type)
                    if garch_model:
                        asset_garch_models[asset] = garch_model
        
        # Estadísticas básicas
        mean_daily = np.mean(portfolio_returns)
        std_daily = np.std(portfolio_returns)
        
        print(f"\n📊 RESUMEN RETORNOS CARTERA:")
        print(f"   • Retorno diario promedio: {mean_daily*100:.4f}%")
        print(f"   • Volatilidad diaria: {std_daily*100:.3f}%")
        print(f"   • Equivalente anual: {mean_daily*252*100:.1f}% retorno, {std_daily*np.sqrt(252)*100:.1f}% volatilidad")
        
        return {
            'returns': portfolio_returns,
            'mean_daily': mean_daily,
            'std_daily': std_daily,
            'portfolio_garch': portfolio_garch,
            'asset_garch_models': asset_garch_models,
            'skewness': stats.skew(portfolio_returns),
            'kurtosis': stats.kurtosis(portfolio_returns),
            'var_95': np.percentile(portfolio_returns, 5),
            'var_99': np.percentile(portfolio_returns, 1),
        }
    
    def monte_carlo_simulation_with_garch(self, weights_dict, returns_data, num_sims=10000):
        """Simulación Monte Carlo usando modelos GARCH para volatilidad condicional"""
        
        print("🔮 Ejecutando simulación Monte Carlo con GARCH...")
        
        # Calcular retornos y ajustar modelos GARCH
        portfolio_metrics = self.calculate_portfolio_returns_with_garch(weights_dict, returns_data)
        historical_returns = portfolio_metrics['returns']
        garch_model = portfolio_metrics['portfolio_garch']
        
        if len(historical_returns) < 50:
            print("⚠️ Pocos datos históricos - usando parámetros conservadores")
            daily_mean, daily_std = 0.0005, 0.012  # ~13% anual retorno, 19% vol
            use_garch = False
        else:
            daily_mean = portfolio_metrics['mean_daily']
            daily_std = portfolio_metrics['std_daily']
            use_garch = garch_model is not None
            
            # Aplicar límites conservadores para casos extremos
            if daily_mean > 0.0015:  # Solo si es mayor a 38% anual
                print(f"   ⚠️ Limitando retorno muy alto de {daily_mean*252*100:.1f}% a 30%")
                daily_mean = 0.0012  # ~30% anual máximo
                
            if daily_std > 0.025:  # Solo si volatilidad es mayor a 40% anual
                print(f"   ⚠️ Limitando volatilidad muy alta de {daily_std*np.sqrt(252)*100:.1f}% a 35%")
                daily_std = 0.022  # ~35% anual máximo
        
        time_horizons = [21, 63, 126, 252]
        results = {}
        all_paths = {}
        
        for days in time_horizons:
            print(f"   📊 Simulando {days} días ({days//21} meses) con {'GARCH' if use_garch else 'distribución normal'}...")
            
            simulation_matrix = np.zeros((num_sims, days))
            
            for sim in range(num_sims):
                if use_garch and days >= 21:
                    # USAR PREDICCIONES GARCH CUANDO ESTÁ DISPONIBLE
                    try:
                        # Predecir volatilidad condicional con GARCH
                        forecast = garch_model.forecast(horizon=min(days, 100), reindex=False)
                        garch_vol = np.sqrt(forecast.variance.values[-1, :min(days, len(forecast.variance.values[-1, :]))])
                        
                        # Si GARCH predice menos días, extender con último valor
                        if len(garch_vol) < days:
                            last_vol = garch_vol[-1]
                            garch_vol = np.concatenate([garch_vol, np.full(days - len(garch_vol), last_vol)])
                        
                        # Generar retornos con volatilidad condicional GARCH
                        garch_returns = []
                        for day in range(days):
                            vol_day = garch_vol[day] / 100  # Convertir de % a decimal
                            ret_day = np.random.normal(daily_mean, vol_day)
                            garch_returns.append(ret_day)
                        
                        simulation_matrix[sim, :] = np.array(garch_returns)
                        
                    except:
                        # Fallback a distribución normal si GARCH falla
                        simulation_matrix[sim, :] = np.random.normal(daily_mean, daily_std, days)
                
                elif len(historical_returns) >= days * 2:
                    # Bootstrap de segmentos históricos para períodos cortos
                    start_idx = np.random.randint(0, len(historical_returns) - days)
                    bootstrap_segment = historical_returns[start_idx:start_idx + days]
                    
                    # Añadir mínimo ruido
                    noise = np.random.normal(0, daily_std * 0.05, days)
                    bootstrap_segment = bootstrap_segment + noise
                    simulation_matrix[sim, :] = bootstrap_segment
                else:
                    # Distribución normal estándar
                    simulation_matrix[sim, :] = np.random.normal(daily_mean, daily_std, days)
                
                # Límites conservadores
                simulation_matrix[sim, :] = np.clip(simulation_matrix[sim, :], -0.06, 0.06)
            
            # Calcular valores finales
            cumulative_returns = np.cumprod(1 + simulation_matrix, axis=1)
            final_values = cumulative_returns[:, -1]
            
            # Retornos para el período específico
            period_returns = final_values - 1
            
            # Estadísticas del período
            percentiles = [1, 5, 10, 25, 50, 75, 90, 95, 99]
            period_percentiles = np.percentile(period_returns, percentiles)
            
            # Probabilidades de pérdida
            prob_loss = np.sum(period_returns < 0) / len(period_returns)
            prob_loss_10_period = np.sum(period_returns < -0.10) / len(period_returns)
            prob_loss_20_period = np.sum(period_returns < -0.20) / len(period_returns)
            prob_loss_30_period = np.sum(period_returns < -0.30) / len(period_returns)
            
            # VaR y CVaR
            var_95_period = np.percentile(period_returns, 5)
            var_99_period = np.percentile(period_returns, 1)
            cvar_95_period = np.mean(period_returns[period_returns <= var_95_period])
            cvar_99_period = np.mean(period_returns[period_returns <= var_99_period])
            
            results[days] = {
                'horizon_months': days // 21,
                'horizon_days': days,
                'period_stats': {
                    'mean': np.mean(period_returns),
                    'std': np.std(period_returns),
                    'min': np.min(period_returns),
                    'max': np.max(period_returns),
                    'percentiles': dict(zip(percentiles, period_percentiles)),
                },
                'probabilities': {
                    'loss': prob_loss,
                    'loss_10': prob_loss_10_period,
                    'loss_20': prob_loss_20_period,
                    'loss_30': prob_loss_30_period
                },
                'risk_metrics': {
                    'var_95': var_95_period,
                    'var_99': var_99_period,
                    'cvar_95': cvar_95_period,
                    'cvar_99': cvar_99_period
                },
                'garch_used': use_garch
            }
            
            # Guardar caminos para visualización
            if days == 252:
                all_paths[days] = cumulative_returns[:100]
        
        return results, all_paths, portfolio_metrics
    
    def stress_testing_argentina(self, weights_dict):
        """Stress testing específico para Argentina"""
        results = {}
        
        for scenario_name, params in self.scenarios_argentina.items():
            portfolio_impact = 0
            
            for asset, weight in weights_dict.items():
                asset_type = self.classify_asset_argentina(asset)
                crisis_sensitivity = self.asset_adjustments[asset_type]['crisis_sensitivity']
                
                # Ajustar shock por tipo de activo
                asset_shock = params['shock'] * crisis_sensitivity
                portfolio_impact += weight * asset_shock
            
            results[scenario_name] = {
                'impact': portfolio_impact,
                'duration_days': params['duration'],
                'probability': params['prob'],
                'severity': abs(portfolio_impact),
                'expected_loss': portfolio_impact * params['prob']
            }
        
        return results

# =============================================================================
# 📊 FUNCIÓN DE ANÁLISIS COMPLETO CON PRESENTACIÓN CORREGIDA
# =============================================================================

def analizar_monte_carlo_argentina_garch(cartera_weights, returns_data):
    """Análisis completo Monte Carlo con modelos GARCH/ARCH"""
    
    if not cartera_weights or returns_data.empty:
        print("❌ ERROR: No hay datos suficientes para el análisis")
        return None
    
    print("🇦🇷 ANÁLISIS MONTE CARLO CON GARCH - CONTEXTO ARGENTINO")
    print("=" * 75)
    
    # Inicializar simulador con GARCH
    mc_simulator = MonteCarloArgentinaARCH(risk_free_rate=0.04)
    
    # Ejecutar simulaciones con GARCH
    mc_results, paths, portfolio_metrics = mc_simulator.monte_carlo_simulation_with_garch(
        cartera_weights, returns_data
    )
    stress_results = mc_simulator.stress_testing_argentina(cartera_weights)
    
    # =============================================================================
    # 📋 1. COMPOSICIÓN DE CARTERA
    # =============================================================================
    
    print("\n📈 COMPOSICIÓN DE TU CARTERA:")
    print("-" * 40)
    
    df_composicion = pd.DataFrame([
        {'Activo': activo, 'Peso (%)': peso*100, 'Tipo': mc_simulator.classify_asset_argentina(activo)}
        for activo, peso in sorted(cartera_weights.items(), key=lambda x: x[1], reverse=True)
        if peso > 0.01
    ])
    
    display(df_composicion.style.format({'Peso (%)': '{:.1f}%'})
                              .background_gradient(subset=['Peso (%)'], cmap='Blues'))
    
    # =============================================================================
    # 🎯 2. INFORMACIÓN MODELOS GARCH AJUSTADOS
    # =============================================================================
    
    print("\n🎯 MODELOS GARCH AJUSTADOS:")
    print("-" * 40)
    
    if portfolio_metrics.get('portfolio_garch'):
        garch_model = portfolio_metrics['portfolio_garch']
        print(f"✅ GARCH de Cartera Completa:")
        print(f"   • Modelo: {garch_model.model.volatility}")
        print(f"   • AIC: {garch_model.aic:.2f}")
        print(f"   • Log-Likelihood: {garch_model.loglikelihood:.2f}")
        
        # Mostrar parámetros del modelo
        print(f"   • Parámetros:")
        for param, value in garch_model.params.items():
            print(f"     - {param}: {value:.6f}")
    else:
        print("⚠️ No se pudo ajustar modelo GARCH para la cartera")
    
    # Información de modelos individuales
    asset_models = portfolio_metrics.get('asset_garch_models', {})
    if asset_models:
        print(f"\n📊 MODELOS GARCH POR ACTIVO PRINCIPAL:")
        for asset, model in asset_models.items():
            print(f"   • {asset}: GARCH ajustado (AIC: {model.aic:.2f})")
    
    # =============================================================================
    # 📊 3. RESULTADOS MONTE CARLO CON GARCH
    # =============================================================================
    
    print(f"\n📊 RESULTADOS SIMULACIÓN MONTE CARLO (CON GARCH):")
    print("-" * 65)
    
    mc_data = []
    for days, results in mc_results.items():
        period_return = results['period_stats']['mean'] * 100
        period_vol = results['period_stats']['std'] * 100
        garch_used = results.get('garch_used', False)
        
        mc_data.append({
            'Horizonte': f"{results['horizon_months']} mes{'es' if results['horizon_months'] > 1 else ''}",
            'Retorno del Período (%)': period_return,
            'Volatilidad del Período (%)': period_vol,
            'VaR 95% del Período (%)': results['risk_metrics']['var_95'] * 100,
            'Prob. Pérdida (%)': results['probabilities']['loss'] * 100,
            'Modelo': 'GARCH' if garch_used else 'Normal'
        })
    
    df_monte_carlo = pd.DataFrame(mc_data)
    
    # Aplicar formato
    styler = df_monte_carlo.style.format({
        'Retorno del Período (%)': '{:+.1f}',
        'Volatilidad del Período (%)': '{:.1f}',
        'VaR 95% del Período (%)': '{:+.1f}',
        'Prob. Pérdida (%)': '{:.1f}',
    })
    
    # Colorear celdas
    def color_returns(val):
        if isinstance(val, (int, float)):
            if val < -10:
                return 'color: darkred; font-weight: bold'
            elif val < 0:
                return 'color: red'
            elif val > 0:
                return 'color: green'
        return 'color: black'
    
    def color_model(val):
        if val == 'GARCH':
            return 'background-color: lightgreen; font-weight: bold'
        else:
            return 'background-color: lightyellow'
    
    styler = styler.applymap(color_returns, subset=['Retorno del Período (%)', 'VaR 95% del Período (%)'])
    styler = styler.applymap(color_model, subset=['Modelo'])
    
    display(styler)
    
    # =============================================================================
    # 💥 4. STRESS TESTING  
    # =============================================================================
    
    print("\n💥 ANÁLISIS DE STRESS TESTING:")
    print("-" * 40)
    
    sorted_stress = sorted(stress_results.items(), 
                          key=lambda x: x[1]['severity'], reverse=True)
    
    stress_data = []
    for scenario, data in sorted_stress:
        stress_data.append({
            'Escenario': scenario,
            'Impacto (%)': data['impact'] * 100,
            'Duración (días)': data['duration_days'],
            'Probabilidad (%)': data['probability'] * 100,
            'Pérdida Esperada (%)': data['expected_loss'] * 100,
            'Severidad': '🔴' if data['severity'] > 0.4 else '🟡' if data['severity'] > 0.2 else '🟢'
        })
    
    df_stress = pd.DataFrame(stress_data)
    
    styler_stress = df_stress.style.format({
        'Impacto (%)': '{:+.1f}',
        'Duración (días)': '{:.0f}',
        'Probabilidad (%)': '{:.1f}',
        'Pérdida Esperada (%)': '{:+.2f}'
    })
    
    styler_stress = styler_stress.background_gradient(subset=['Impacto (%)'], cmap='Reds_r')
    styler_stress = styler_stress.background_gradient(subset=['Probabilidad (%)'], cmap='YlOrRd')
    
    display(styler_stress)
    
    # =============================================================================
    # 📈 5. INTERPRETACIÓN CON GARCH
    # =============================================================================
    
    print(f"\n📈 INTERPRETACIÓN CON MODELOS GARCH:")
    print("-" * 50)
    
    for days in [21, 63, 126, 252]:
        if days in mc_results:
            months = days // 21
            
            period_return = mc_results[days]['period_stats']['mean'] * 100
            period_vol = mc_results[days]['period_stats']['std'] * 100
            prob_perdida = mc_results[days]['probabilities']['loss'] * 100
            var_95 = mc_results[days]['risk_metrics']['var_95'] * 100
            garch_used = mc_results[days].get('garch_used', False)
            
            print(f"\n🔹 {months} mes{'es' if months > 1 else ''} ({days} días) {'📊 GARCH' if garch_used else '📈 Normal'}:")
            print(f"   • Retorno esperado del período: {period_return:+.1f}%")
            print(f"   • Volatilidad del período: {period_vol:.1f}%")
            print(f"   • Probabilidad de pérdida: {prob_perdida:.1f}%")
            print(f"   • VaR 95% (pérdida): {abs(var_95):.1f}%")
            
            if garch_used:
                print(f"   ✅ Simulación con volatilidad condicional GARCH")
            else:
                print(f"   📊 Simulación con distribución normal estándar")
            
            # Interpretación específica realista
            if period_return < -5:
                print(f"   🔴 Retorno negativo esperado - revisar estrategia")
            elif period_return < 2 and months == 1:
                print(f"   🟡 Retorno mensual bajo")
            elif period_return < 5 and months == 3:
                print(f"   🟡 Retorno trimestral bajo")
            elif period_return < 8 and months == 6:
                print(f"   🟡 Retorno semestral bajo")  
            elif period_return < 15 and months == 12:
                print(f"   🟡 Retorno anual bajo")
            else:
                print(f"   ✅ Retorno esperado razonable para el período")
                
            if prob_perdida > 50:
                print(f"   ⚠️ Alta probabilidad de pérdida")
            elif prob_perdida > 35:
                print(f"   📊 Probabilidad moderada de pérdida")
            else:
                print(f"   ✅ Baja probabilidad de pérdida")
    
    # =============================================================================
    # 📊 6. VISUALIZACIONES
    # =============================================================================
    
    create_visualizations_argentina(mc_results, paths, stress_results)
    
    return mc_results, stress_results, portfolio_metrics

# =============================================================================
# 🎨 FUNCIÓN DE VISUALIZACIONES MEJORADAS
# =============================================================================

def create_visualizations_argentina(mc_results, paths, stress_results):
    """Crea visualizaciones específicas para Argentina"""
    
    # 1. Distribuciones de retorno por horizonte (usando retornos del período)
    fig_dist = make_subplots(
        rows=2, cols=2,
        subplot_titles=['1 Mes', '3 Meses', '6 Meses', '1 Año'],
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    horizons = [21, 63, 126, 252]
    positions = [(1,1), (1,2), (2,1), (2,2)]
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']
    
    for i, (days, pos, color) in enumerate(zip(horizons, positions, colors)):
        if days in mc_results:
            mean_ret = mc_results[days]['period_stats']['mean']
            std_ret = mc_results[days]['period_stats']['std']
            
            # Crear curva de distribución
            x_vals = np.linspace(mean_ret - 3*std_ret, mean_ret + 3*std_ret, 100)
            y_vals = stats.norm.pdf(x_vals, mean_ret, std_ret)
            
            fig_dist.add_trace(
                go.Scatter(x=x_vals*100, y=y_vals, 
                          fill='tozeroy', fillcolor=color, opacity=0.7,
                          name=f'{days//21} Meses', line=dict(color=color, width=2)),
                row=pos[0], col=pos[1]
            )
            
            # Líneas de percentiles
            p5 = mc_results[days]['period_stats']['percentiles'][5] * 100
            p95 = mc_results[days]['period_stats']['percentiles'][95] * 100
            
            fig_dist.add_vline(x=p5, line_dash="dash", line_color="red", 
                              row=pos[0], col=pos[1])
            fig_dist.add_vline(x=p95, line_dash="dash", line_color="green", 
                              row=pos[0], col=pos[1])
    
    fig_dist.update_layout(
        title='🇦🇷 Distribución de Retornos por Período - Contexto Argentino',
        height=600, showlegend=False, template='plotly_white'
    )
    fig_dist.update_xaxes(title_text="Retorno del Período (%)")
    fig_dist.update_yaxes(title_text="Densidad de Probabilidad")
    fig_dist.show()
    
    # 2. Stress Testing - Gráfico de barras
    if stress_results:
        scenarios = list(stress_results.keys())
        impacts = [stress_results[s]['impact'] * 100 for s in scenarios]
        probabilities = [stress_results[s]['probability'] * 100 for s in scenarios]
        
        # Colorear por severidad
        colors_stress = []
        for imp in impacts:
            if imp < -40:
                colors_stress.append('#8B0000')  # Rojo oscuro
            elif imp < -25:
                colors_stress.append('#FF0000')  # Rojo
            elif imp < -15:
                colors_stress.append('#FF8C00')  # Naranja
            else:
                colors_stress.append('#FFD700')  # Amarillo
        
        fig_stress = go.Figure(go.Bar(
            x=impacts,
            y=scenarios,
            orientation='h',
            marker_color=colors_stress,
            text=[f'{imp:+.1f}% (P:{prob:.0f}%)' 
                  for imp, prob in zip(impacts, probabilities)],
            textposition='auto',
            hovertemplate='%{y}<br>Impacto: %{x:+.1f}%<extra></extra>'
        ))
        
        fig_stress.update_layout(
            title='💥 Stress Testing - Escenarios Argentinos',
            xaxis_title='Impacto en Cartera (%)',
            yaxis_title='Escenario',
            template='plotly_white',
            height=500
        )
        fig_stress.add_vline(x=0, line_dash="dash", line_color="black")
        fig_stress.show()
        
    # Guardar gráficos
    try:
        fig_dist.write_image("monte_carlo_distribuciones_periodo.png", width=1200, height=600, scale=2)
        if 'fig_stress' in locals():
            fig_stress.write_image("stress_testing_argentina_final.png", width=1000, height=500, scale=2)
        print("📁 Gráficos guardados exitosamente")
    except:
        print("⚠️ No se pudieron guardar los gráficos (requiere kaleido)")

# =============================================================================
# 🚀 EJECUTAR ANÁLISIS MONTE CARLO CON GARCH
# =============================================================================

if 'cartera_real_weights' in globals() and cartera_real_weights and 'df_data_completo' in globals():
    print("🚀 Iniciando análisis Monte Carlo con GARCH para Argentina...")
    mc_results, stress_results, portfolio_metrics = analizar_monte_carlo_argentina_garch(
        cartera_real_weights, df_data_completo
    )
    
    if mc_results:
        print("\n🎯 RESUMEN FINAL CON GARCH:")
        print("-" * 50)
        print("RETORNOS ESPERADOS POR PERÍODO (con volatilidad condicional GARCH):")
        for days in [21, 63, 126, 252]:
            if days in mc_results:
                period_ret = mc_results[days]['period_stats']['mean'] * 100
                months = days // 21
                garch_used = mc_results[days].get('garch_used', False)
                model_str = "GARCH" if garch_used else "Normal"
                print(f"   {months} mes(es): {period_ret:+.1f}% del capital [{model_str}]")
        
        print(f"\n✅ Análisis Monte Carlo completado con modelos GARCH")
        print("📝 Volatilidad condicional modelada con ARCH/GARCH según disponibilidad de datos")
        
        # Información adicional sobre GARCH
        if portfolio_metrics.get('portfolio_garch'):
            print(f"🎯 Modelo GARCH de cartera ajustado correctamente")
        else:
            print(f"⚠️ Modelo GARCH no disponible - usando distribución normal calibrada")
            
    else:
        print("❌ Error en el análisis")
else:
    print("❌ ERROR: Faltan datos de cartera o precios históricos")
    mc_results = stress_results = portfolio_metrics = None

# =============================================================================
# 🎨 VISUALIZACIONES MONTE CARLO CON GARCH - AGREGADAS
# =============================================================================

if 'mc_results' in locals() and mc_results is not None:
    print(f"\n🎨 GENERANDO VISUALIZACIONES MONTE CARLO CON GARCH...")
    
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    import numpy as np
    
    # 1. DISTRIBUCIONES DE RETORNOS POR HORIZONTE
    fig_dist = make_subplots(
        rows=2, cols=2,
        subplot_titles=['1 Mes (GARCH)', '3 Meses (GARCH)', '6 Meses (GARCH)', '12 Meses (GARCH)'],
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
    horizons = [21, 63, 126, 252]
    positions = [(1, 1), (1, 2), (2, 1), (2, 2)]
    
    for i, (days, pos, color) in enumerate(zip(horizons, positions, colors)):
        if days in mc_results:
            # Generar distribución simulada para visualización
            stats = mc_results[days]['period_stats']
            mean_ret = stats['mean']
            std_ret = stats['std']
            
            # Simulación rápida para histograma
            sample_returns = np.random.normal(mean_ret, std_ret, 1000)
            
            fig_dist.add_trace(
                go.Histogram(
                    x=sample_returns * 100,
                    name=f'{days//21} mes(es)',
                    nbinsx=30,
                    marker_color=color,
                    opacity=0.7
                ),
                row=pos[0], col=pos[1]
            )
    
    fig_dist.update_layout(
        title_text="🔮 Distribuciones de Retornos Monte Carlo con GARCH",
        showlegend=False,
        height=600
    )
    
    fig_dist.update_xaxes(title_text="Retorno del Período (%)")
    fig_dist.update_yaxes(title_text="Frecuencia")
    
    fig_dist.show()
    
    # 2. EVOLUCIÓN TEMPORAL DE CAMINOS MONTE CARLO (252 días)
    if 'paths' in locals() and paths and 252 in paths and paths[252] is not None:
        fig_paths = go.Figure()
        
        # Mostrar solo los primeros 20 caminos para claridad
        sample_paths = paths[252][:20]
        
        for i, path in enumerate(sample_paths):
            valor_acumulado = np.cumprod(1 + path) * 100  # Convertir a valor inicial 100
            
            fig_paths.add_trace(
                go.Scatter(
                    x=list(range(len(valor_acumulado))),
                    y=valor_acumulado,
                    mode='lines',
                    name=f'Camino {i+1}',
                    line=dict(width=1, color=f'rgba({np.random.randint(0,255)},{np.random.randint(0,255)},{np.random.randint(0,255)},0.6)'),
                    showlegend=False
                )
            )
        
        # Agregar línea de valor inicial
        fig_paths.add_hline(y=100, line_dash="dash", line_color="black", annotation_text="Valor Inicial")
        
        fig_paths.update_layout(
            title="🛤️ Caminos Monte Carlo - Evolución de Cartera (12 meses con GARCH)",
            xaxis_title="Días Hábiles",
            yaxis_title="Valor de Cartera (Base 100)",
            height=500
        )
        
        fig_paths.show()
    
    # 3. HEATMAP DE MÉTRICAS DE RIESGO
    horizons_labels = ["1 mes", "3 meses", "6 meses", "12 meses"]
    metrics_labels = ["Retorno %", "Volatilidad %", "VaR 95%", "Prob. Pérdida %"]
    
    # Crear matriz de datos
    heatmap_data = []
    for days in horizons:
        if days in mc_results:
            stats = mc_results[days]['period_stats']
            var_95 = mc_results[days]['risk_metrics']['var_95']
            prob_loss = mc_results[days]['probabilities']['loss']
            
            row_data = [
                stats['mean'] * 100,      # Retorno %
                stats['std'] * 100,       # Volatilidad %
                abs(var_95) * 100,        # VaR 95% (valor absoluto)
                prob_loss * 100           # Probabilidad pérdida %
            ]
            heatmap_data.append(row_data)
    
    if heatmap_data:
        fig_heatmap = go.Figure(data=go.Heatmap(
            z=heatmap_data,
            x=metrics_labels,
            y=horizons_labels,
            colorscale='RdYlBu_r',
            text=[[f"{val:.1f}" for val in row] for row in heatmap_data],
            texttemplate="%{text}",
            textfont={"size": 12}
        ))
    
        fig_heatmap.update_layout(
            title="🌡️ Mapa de Calor: Métricas de Riesgo por Horizonte Temporal (GARCH)",
            height=400
        )
        
        fig_heatmap.show()
    
    # 4. STRESS TESTING VISUAL
    if 'stress_results' in locals() and stress_results:
        scenarios_names = list(stress_results.keys())
        impacts = [abs(data['impact']) * 100 for data in stress_results.values()]
        probabilities = [data['probability'] * 100 for data in stress_results.values()]
    
        fig_stress = go.Figure()
    
        # Gráfico de barras de impactos
        fig_stress.add_trace(go.Bar(
            name='Impacto (%)',
            x=scenarios_names,
            y=impacts,
            yaxis='y',
            marker_color='red',
            opacity=0.7
        ))
    
        # Crear segundo eje Y para probabilidades
        fig_stress.add_trace(go.Scatter(
            name='Probabilidad (%)',
            x=scenarios_names,
            y=probabilities,
            yaxis='y2',
            mode='lines+markers',
            line=dict(color='blue', width=3),
            marker=dict(size=8, color='blue')
        ))
    
        fig_stress.update_layout(
            title='💥 Análisis de Stress Testing - Argentina',
            xaxis=dict(title='Escenarios de Crisis'),
            yaxis=dict(title='Impacto Negativo (%)', side='left', color='red'),
            yaxis2=dict(title='Probabilidad (%)', side='right', overlaying='y', color='blue'),
            height=500
        )
    
        fig_stress.update_xaxes(tickangle=45)
        fig_stress.show()
    
    # Guardar gráficos
    try:
        fig_dist.write_image("monte_carlo_distribuciones_garch.png", width=1200, height=600, scale=2)
        if 'fig_paths' in locals():
            fig_paths.write_image("monte_carlo_caminos_garch.png", width=1000, height=500, scale=2)
        if 'fig_heatmap' in locals():
            fig_heatmap.write_image("monte_carlo_heatmap_garch.png", width=800, height=400, scale=2)
        if 'fig_stress' in locals():
            fig_stress.write_image("stress_testing_garch.png", width=1000, height=500, scale=2)
        print("📁 Gráficos guardados exitosamente")
    except:
        print("⚠️ No se pudieron guardar los gráficos (requiere kaleido)")
    
    print("🎨 ¡VISUALIZACIONES AGREGADAS EXITOSAMENTE!")
else:
    print("⚠️ No hay resultados Monte Carlo para visualizar")

🚀 Iniciando análisis Monte Carlo con GARCH para Argentina...
🇦🇷 ANÁLISIS MONTE CARLO CON GARCH - CONTEXTO ARGENTINO
🔮 Ejecutando simulación Monte Carlo con GARCH...
📊 Calculando retornos y ajustando modelos GARCH por activo...

🔮 AJUSTE DE MODELO GARCH PARA CARTERA COMPLETA:
   ✅ Modelo GARCH(1,1) ajustado exitosamente
   📊 AIC: 604.54, Log-Likelihood: -298.27

📈 AJUSTE DE MODELOS GARCH POR ACTIVO:
   🔸 SPY (international_etf):
   ✅ Modelo GARCH(1,1) ajustado exitosamente
   📊 AIC: 608.20, Log-Likelihood: -300.10
   🔸 EWZ (international_etf):
   ✅ Modelo GARCH(1,1) ajustado exitosamente
   📊 AIC: 685.38, Log-Likelihood: -338.69
   🔸 AAPL (us_stocks):
   ✅ Modelo GARCH(1,1) ajustado exitosamente
   📊 AIC: 751.09, Log-Likelihood: -371.55

📊 RESUMEN RETORNOS CARTERA:
   • Retorno diario promedio: 0.1459%
   • Volatilidad diaria: 1.636%
   • Equivalente anual: 36.8% retorno, 26.0% volatilidad
   📊 Simulando 21 días (1 meses) con GARCH...

🔮 AJUSTE DE MODELO GARCH PARA CARTERA COMPLETA:
   

,Activo,Peso (%),Tipo
0,SPY,52.9%,international_etf
1,EWZ,24.0%,international_etf
2,AAPL,18.9%,us_stocks
3,METRO,1.8%,us_stocks
4,CEPU,1.7%,us_stocks



🎯 MODELOS GARCH AJUSTADOS:
----------------------------------------
✅ GARCH de Cartera Completa:
   • Modelo: GARCH(p: 1, q: 1)
   • AIC: 604.54
   • Log-Likelihood: -298.27
   • Parámetros:
     - mu: 0.219430
     - omega: 0.124473
     - alpha[1]: 0.154422
     - beta[1]: 0.797034

📊 MODELOS GARCH POR ACTIVO PRINCIPAL:
   • SPY: GARCH ajustado (AIC: 608.20)
   • EWZ: GARCH ajustado (AIC: 685.38)
   • AAPL: GARCH ajustado (AIC: 751.09)

📊 RESULTADOS SIMULACIÓN MONTE CARLO (CON GARCH):
-----------------------------------------------------------------


,Horizonte,Retorno del Período (%),Volatilidad del Período (%),VaR 95% del Período (%),Prob. Pérdida (%),Modelo
0,1 mes,+3.1,7.4,-8.5,35.2,GARCH
1,3 meses,+9.5,13.8,-11.9,25.4,GARCH
2,6 meses,+19.8,21.3,-11.6,17.6,GARCH
3,12 meses,+44.6,37.1,-7.6,9.0,GARCH



💥 ANÁLISIS DE STRESS TESTING:
----------------------------------------


,Escenario,Impacto (%),Duración (días),Probabilidad (%),Pérdida Esperada (%),Severidad
0,Crisis Global + Local,-68.4,200,3.0,-2.05,🔴
1,Default Soberano,-58.6,365,8.0,-4.69,🔴
2,Crisis Cambiaria,-48.8,90,15.0,-7.33,🔴
3,Crisis Política,-39.1,120,10.0,-3.91,🟡
4,Devaluación Fuerte,-34.2,60,20.0,-6.84,🟡
5,Hiperinflación,-29.3,252,5.0,-1.47,🟡
6,Recesión Local,-24.4,180,25.0,-6.11,🟡



📈 INTERPRETACIÓN CON MODELOS GARCH:
--------------------------------------------------

🔹 1 mes (21 días) 📊 GARCH:
   • Retorno esperado del período: +3.1%
   • Volatilidad del período: 7.4%
   • Probabilidad de pérdida: 35.2%
   • VaR 95% (pérdida): 8.5%
   ✅ Simulación con volatilidad condicional GARCH
   ✅ Retorno esperado razonable para el período
   📊 Probabilidad moderada de pérdida

🔹 3 meses (63 días) 📊 GARCH:
   • Retorno esperado del período: +9.5%
   • Volatilidad del período: 13.8%
   • Probabilidad de pérdida: 25.4%
   • VaR 95% (pérdida): 11.9%
   ✅ Simulación con volatilidad condicional GARCH
   ✅ Retorno esperado razonable para el período
   ✅ Baja probabilidad de pérdida

🔹 6 meses (126 días) 📊 GARCH:
   • Retorno esperado del período: +19.8%
   • Volatilidad del período: 21.3%
   • Probabilidad de pérdida: 17.6%
   • VaR 95% (pérdida): 11.6%
   ✅ Simulación con volatilidad condicional GARCH
   ✅ Retorno esperado razonable para el período
   ✅ Baja probabilidad de pér

📁 Gráficos guardados exitosamente

🎯 RESUMEN FINAL CON GARCH:
--------------------------------------------------
RETORNOS ESPERADOS POR PERÍODO (con volatilidad condicional GARCH):
   1 mes(es): +3.1% del capital [GARCH]
   3 mes(es): +9.5% del capital [GARCH]
   6 mes(es): +19.8% del capital [GARCH]
   12 mes(es): +44.6% del capital [GARCH]

✅ Análisis Monte Carlo completado con modelos GARCH
📝 Volatilidad condicional modelada con ARCH/GARCH según disponibilidad de datos
🎯 Modelo GARCH de cartera ajustado correctamente

🎨 GENERANDO VISUALIZACIONES MONTE CARLO CON GARCH...


📁 Gráficos guardados exitosamente
🎨 ¡VISUALIZACIONES AGREGADAS EXITOSAMENTE!
